In [55]:
import os
import re
import json
import gzip
import time
import zipfile
import requests
import numpy as np
import pandas as pd
from datetime import date
from decimal import Decimal
#from tokens import YANDEX_GEO # Раскоменить для работы с API Яндекс.Организации и получения координат
from tokens import DATA_MOS_RU # Раскоменить для работы с API data.mos.ru
#from tokens import MAPS_CO
MAPS_CO = 0 # # Закоментить строку если есть ключ
YANDEX_GEO = 0 # Закоментить строку если есть ключ
COUNTER = 0# глобальный счетчик, можно использовать для подсчета кол-ва запросов по АПИ
LIMIT = 1000 # лимит запросов по АПИ и кол-во записей в одно запросе
CACHE = {} # глобальный кэш, в процессе скрейпинга не обновлять!
SETS = './sets/' # папка с файлами с наборами

In [2]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_cache(filename='cache.gz'): # загрузка кэша
    global CACHE
    if os.path.exists(filename) and len(CACHE)==0:
        with gzip.open(filename, 'rb') as gzip_ref:
            CACHE = json.load(gzip_ref)
        print(f'Загружено {len(CACHE)} записей из {filename} в кэш')


def save_cache(filename='cache.gz'): # сохранение кэша
    global CACHE
    with gzip.open(filename, 'wb') as gzip_file:
        json_data = json.dumps(CACHE, ensure_ascii=False, default=default_serializer)
        gzip_file.write(json_data.encode('utf-8'))
    print(f'Сохранено {len(CACHE)} записей в {filename}')
    
    
def load_gzon(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return json.load(gzip_ref)
    else:
        return []
    

def save_gzon(filename):
    data = globals().get(filename)
    if data is not None:
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(json_data)} записей в {filename}.gz')
    else:
        print(f'Переменная {filename} не найдена')
        
     
def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')
        
    
def mos_geojson(dataset):
    all_data = load_gzon(dataset)
    if(len(all_data) > 0):
        return all_data

    step = LIMIT
    url = "https://apidata.mos.ru/v1"
    folder = "/datasets/"
    params = {
        "api_key": DATA_MOS_RU,
        "$skip": 0
    }
    
    count = int(requests.get(url + folder + dataset + "/count", params=params).text)
    while count > params["$skip"]:
        response = requests.get(url + folder + dataset + "/rows", params=params)
        data = json.loads(response.text)
        for d in data:
            all_data.append(d['Cells'])
        params["$skip"] += step
    json_data = json.dumps(all_data, ensure_ascii=False)
    with gzip.open(SETS + dataset + '.gz', 'wb') as gzip_file:
        gzip_file.write(json_data.encode('utf-8'))
    print(f'Сохранено {len(all_data)} записей в {dataset}.gz')
        
    return all_data
           

def get_dataset(url, encoding='utf-8', forma='geojson')->pd.DataFrame:
    if(forma == 'geojson'):
        dataset = pd.DataFrame(mos_geojson(url))
    else:
        filename = ''
        filenames = []
        response = requests.get(url) 
        if response.status_code == 200:
            content_disposition = response.headers.get("Content-Disposition")
            if content_disposition:
                filename = re.findall("filename=(.+)", content_disposition)
                if filename:
                    filename = filename[0]
                    filename = re.sub(r'"', '', filename)
                    with open(filename, "wb") as file:
                        file.write(response.content)
                else:
                    print("Не удалось получить имя файла")
            else:
                print("Не удалось получить информацию о загруженном файле")
        else:
            print("Ошибка при загрузке файла")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            filenames = zip_ref.namelist()
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extract(filenames[0])
        if( forma == 'csv'):
            dataset = pd.read_csv(filenames[0], sep=';', encoding=encoding)
            dataset = dataset.drop(index=0)
            dataset = dataset.drop(columns=[col for col in dataset.columns if col.startswith("Unnamed:")])
        elif(forma == 'json'):
            dataset = pd.read_json(filenames[0], encoding=encoding)
        os.remove(filename)
        os.remove(filenames[0])
    print(dataset.info())
    return dataset

def fill_ecology(collector, emitter_str, filter_col, filter_val, start, end, geodata, res_start=None, res_end=None):    
    """
    Функция возвращает датасет с 3 колонками: начало события, его конец, и названине колонки события
    Колонка события будет содержат 1 в случае экологически значемых случаев.
    Данный случаи мы фильтруем из оригинального датасета (data) с помощью filter_col и filter_val/
    Параметры:
    - collector: pandas.DataFrame, большой датасет в который записываем данные из маленького
    - emitter_str: str, мальенький датасет из которого берем события для экспорта передаем в кавычках
    - filter_col: str, название столбца для фильтрации, как правило это столбец с результатами замеров экологии
    - filter_val: str, значения определяем сами, например ключевое слово выявлен, найдет все выявленные случаи
    - start: str, дата начала события
    - end: str, дата конца события
    - geodata: str, геоданные
    - res_start: str, дата начала резервной даты начала, на случай если в изночальном столбце не будет данных
    - res_end: str, дата конца резервной даты конца, на случай если в изночальном столбце не будет данных

    Возвращает:
    - pandas.DataFrame, отфильтрованные и переименованные данные для экспорта в большой экологический датасет
    """
    if(res_start is None):
        res_start = start
    if(res_end is None):
        res_end = end
    emitter = globals()[emitter_str] 
    temp = emitter[emitter[filter_col].str.contains(filter_val, case=False)][[start, end, geodata, res_start, res_end]]
    temp.columns = ['start', 'end', emitter_str, 'res_start', 'res_end']
    temp[['start', 'end', 'res_start', 'res_end']] = temp[['start', 'end', 'res_start', 'res_end']].astype(str)
    temp['start'] = temp.apply(lambda x: x['res_start'] if len(x['start']) < 1 else x['start'], axis=1)
    temp['end'] = temp.apply(lambda x: x['res_end'] if len(x['end']) < 1 else x['end'], axis=1)
    temp = temp.drop(columns=['res_start', 'res_end'])
    collector = pd.concat([collector, temp])
    return collector


def maps_co(address):
    global CACHE
    global LIMIT
    url = "https://geocode.maps.co/search"
    params = {
        "api_key": MAPS_CO,
        "q": ""
    }
    #for address in addresses:
       # data = []
    if address in CACHE:
        data = CACHE[address]
    else:
        time.sleep(1)
        params["q"] = address
        try:
            response = requests.get(url, params=params)
        except:
            return None
        try:
            data = response.json()[0]['boundingbox']
        except:
            data = None
        if(len(CACHE)%LIMIT)==0: # проверяем увеличился ли кэш на тысячу записей
            save_cache('cache.gz') 
        CACHE[address] = data
    
    return {'boundingbox': data}


def yandex_geo(address):
    global CACHE
    global LIMIT
    
    key = address
    if key in geo_cache:
        return geo_cache[key]
    
    url = "https://geocode-maps.yandex.ru/1.x"
    params = {
        "apikey": YANDEX_GEO,
        "lang": "ru_RU",
        "format": "json",
        "geocode": key
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'response' in data and 'GeoObjectCollection' in data['response']:
            geo_object_collection = data['response']['GeoObjectCollection']
            if 'featureMember' in geo_object_collection and len(geo_object_collection['featureMember']) > 0:
                geo_object = geo_object_collection['featureMember'][0]['GeoObject']
                if 'Point' in geo_object and 'pos' in geo_object['Point']:
                    geo_data = {'Point': {'pos': geo_object['Point']['pos']}}
                    geo_cache[key] = geo_data
                    if(len(CACHE)%LIMIT)==0: # проверяем увеличился ли кэш на тысячу записей
                        save_cache('cache.gz') # сохраняем кэш (бэкап)
                    return geo_data
        
        geo_cache[key] = None
        return None
    except Exception as e:
        print(f"Ошибка при запросе к API: {e}")
        geo_cache[key] = None
        return None


def to_geoData(dataset, address_col):
    global CACHE
    dataset['geoData'] = None
    load_cache('cache.gz')
    dataset['geoData'] = dataset[address_col].apply(lambda x: to_geo(x))
    # Сохранение кэша в файл:
    save_cache('cache.gz')
        
    return dataset


def to_geo(unknown):
    geo = []
    if(type(unknown) == str):
        geo = maps_co(short_address(unknown))
        if(len(geo)==0):
            geo = yandex_geo(unknown)
    if(type(unknown) == list):
        for address in unknown:
            geo.append(maps_co(short_address(address['Address'])))
            if(len(geo)==0):
                geo.append(yandex_geo(address['Address']))
    return geo
    

def obj_to_address(obj):
    addresses = []
    for address in obj:
        addresses.append((address['Address']).split(', '))
    return addresses

def short_address(address):
    city = 'Москва '
    if ('Российская Федерация' in address):
        full_list = address.split(', ')
        for i in full_list:
            if ('город ' in i):
                city = i.replace('город ', '') +' '
        return city + ' '.join(address.split(', ')[3:]) \
                .replace('дом ', '').replace('корпус', 'к').replace('строение', 'с') \
                .replace('сооружение', 'с').replace('владение', 'вл')
    elif ('город ' in address):
        full_list = address.split(', ')
        city = full_list[0].replace('город ', '')+ ' '
        return city + ' '.join(full_list[1:]).replace('дом ', '').replace('корпус', 'к').replace('строение', 'с') \
                .replace('сооружение', 'с').replace('владение', 'вл').replace(',', '')
    return city + address.replace('дом ', '').replace('корпус', 'к').replace('строение', 'с') \
                .replace('сооружение', 'с').replace('владение', 'вл').replace(',', '')

In [3]:
pd.set_option('display.max_colwidth', None) # для отображения полного текста в ячейках
ecology = pd.DataFrame() # Всеобъемлющий датасет по экологии

# Экологические датасеты

## Table

### Таблица ключевых параметров экологических датасетов

Датасеты имеют разную структуру, разные имена полей. 
Последний датасет вообще не пригоден для использования eco_build_houses, поэтому он не включен в таблицу ниже. В нем отсутствуют не только геданные, но и адрес и дата конца постройки и не понятно про дату начала.

Зеленые строки говорят о том, что предварительно датасет пригоден для использования

Желтые предполагают, что они условно пригодны для использвоания, даже без модификации

Красные строки, предполагают глубокую модификацию данных

<table>
  <tr>
    <th>№</th><th>Датасет</th><th>Дата начла</th><th>Дата конца</th><th>Событие</th><th>Координаты</th><th>Ключ</th>
  </tr>
    <tr style="color: green;"><td>1</td><td><a href=#Air>air</a></td><td>Date</td><td>Date</td><td>Results</td><td>geoData</td><td>выявлен</td></tr>
    <tr style="color: green;"><td>2</td><td><a href=#Mean>mean</a></td><td>Period</td><td>Period</td><td>Period</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>3</td><td><a href=#Noise>noise</a></td><td>Date</td><td>Date</td><td>Results</td><td>geoData</td><td>выявлен</td></tr>
  <tr style="color: green;"><td>4</td><td><a href=#Dirt>dirt</a></td><td>Year</td><td>Year</td><td>PollutionCategory</td><td>geoData</td><td>опасная</td></tr>
  <tr style="color: green;"><td>5</td><td><a href=#Digging>digging</a></td><td>WorkStartDate (Date)</td><td>WorkEndDate</td><td>WorkStartDate</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>6</td><td><a href=#Capital>capital</a></td><td>StartDateActual (StarDate)</td><td>EndDateActual (EndDate)</td><td>StartDateActual</td><td>ObjectAddress (geoData)</td><td>''</td></tr>
  <tr style="color: green;"><td>7</td><td><a href=#House_lines>house_lines</a></td><td>ActualStartDate</td><td>ActualEndDate</td><td>WorkReason</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>8</td><td><a href=#Clinics>clinics</a></td><td>ActualStartDate (PlannedStartDate)</td><td>ActualEndDate (PlannedEndDate)</td><td>PlannedEndDate</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>9</td><td><a href=#Roads>roads</a></td><td>ActualBeginDate (WorksBeginDate)</td><td>ActualEndDate (PlannedEndDate)</td><td>PlannedEndDate</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>10</td><td><a href=#Parks>parks</a></td><td>StartDateActual (StartDatePlanned)</td><td>EndDateActual (EndDatePlanned)</td><td>EndDatePlanned</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>11</td><td><a href=#Transports>trasports</a></td><td>ActualStartDate (PlannedStartDate)</td><td>ActualEndDate (PlannedEndDate)</td><td>PlannedEndDate</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>12</td><td><a href=#City_places>city_places</a></td><td>StartDateActual (StartDate)</td><td>EndDateActual (EndDate)</td><td>StartDateActual</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>13</td><td><a href=#Live_area>live_area</a></td><td>StartDateActual (StartDate)</td><td>EndDateActual (EndDate)</td><td>StartDateActual</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>14</td><td><a href=#Streets>streets</a></td><td>ActualStartDate (PlannedStartDate)</td><td>ActualEndDate (PlannedEndDate)</td><td>PlannedEndDate</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>15</td><td><a href=#Trashes>trashes</a></td><td>start</td><td>end</td><td>YardName</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>16</td><td><a href=#Factories>factories</a></td><td>start</td><td>end</td><td>Category</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>17</td><td><a href=#Markets>markets</a></td><td>start</td><td>end</td><td>MarketType</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>18</td><td><a href=#Autostations>autostations</a></td><td>start</td><td>end</td><td>Name</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>19</td><td><a href=#Gasstations>gasstations</a></td><td>start</td><td>end</td><td>Owner</td><td>geoData</td><td>''</td></tr>
  <tr style="color: green;"><td>20</td><td><a href=#Bad_gasstations>bad_gasstations</a></td><td>start(TestDate)</td><td>end(TestDate)</td><td>Owner(Violations)</td><td>geoData</td><td>''</td></tr>
    <tr style="color: green;"><td>21</td><td><a href=#Buildgrounds>buildgrounds</a></td><td>GPZUDocumentDate</td><td>end</td><td>GPZUDocumentDate</td><td>geoData</td><td>''</td></tr>
</table>

* '' - пусто, поиск любого события, т.к. нет градации и сам факт события это эгологический фактор

## Air

ID:2457

**Общее загрязнение атмосферы** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/2457)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=218714)

Набор данных позволяет получить информацию о месте отбора проб атмосферного воздуха, дате проведения отбора, основных результатах исследования. Для удобства ориентирования предусмотрена возможность просмотра местоположения контрольной точки исследования атмосферного воздуха на карте города Москвы.

В 2015 году Министерством природных ресурсов и экологии РФ был составлен экологический рейтинг российских городов, при составлении которого учитывались данные о состоянии атмосферного воздуха, водопользования, по выбросам от транспорта, об использовании городских территорий и энергопотреблении. По результатам рейтинга Москва - лидер по экологическому развитию среди остальных российских городов.

В городе Москве организована уникальная система мониторинга загрязнения атмосферного воздуха, в состав которой входят, кроме стационарных станций контроля загрязнения атмосферного воздуха, три передвижные лаборатории ГПБУ «Мосэкомониторинг».

Передвижная лаборатория – это автомобиль, оснащенный современным оборудованием, которое позволяет измерять загрязняющие вещества в режиме реального времени по основным загрязняющим веществам и отбирать пробы воздуха для последующего анализа в лаборатории на расширенный перечень веществ.

В Российской Федерации установлены гигиенические нормативы предельно допустимых концентраций (ПДК) загрязняющих веществ в атмосферном воздухе. В случае выявления в ходе рейдов передвижной лаборатории превышений нормативов информация о зафиксированных нарушениях направляется в соответствующие органы исполнительной власти для принятия мер.

Много сил, труда и терпения   потребовалось ученым, чтобы доказать, что воздух - не самостоятельное вещество, а сложная смесь газов. Впервые высказался о сложном составе воздуха ученый-художник Леонардо да Винчи (XV век).

Количественный состав воздуха  установил французский ученый Антуан Лоран Лавуазье. По результатам своих опытов он сделал вывод, что воздух в целом состоит из кислорода, пригодного для дыхания и горения, и азота в пропорциях 1/5 и 4/5 объема соответственно.

В настоящее время известно, что атмосферный воздух – это смесь газов, состоящая из постоянных и переменных компонентов. На высотах до 100 км относительный состав газов составляет: азот 78%, кислород 21%, около 1% инертного газа аргона, вклад остальных постоянных компонентов существенно меньше.

К переменным компонентам относится водяной пар, озон, двуокись углерода, диоксид серы, диоксид азота и другие, их доля составляет сотые и тысячные доли процентов и они называются «малыми» газами. Кроме этого, в атмосфере присутствуют взвешенные твердые частицы и аэрозоли.

Изучение загрязнения атмосферного воздуха началось с развитием промышленности и ростом городов.

В России одним из первых на вред промышленных выбросов в 1851 году обратил внимание Д.И.Менделеев в статье «О происхождении дыма», в которой писал: «Дым затемняет день, проникает в жилище, грязнит фасады зданий и общественные фабрики, причиняет многие неудобства и нездоровья».

Во второй половине XIX века в таком крупном индустриальном городе, как Чикаго началась борьба с дымом, в рамках которой появление густого дыма было объявлено нарушением общественного порядка, за которое причитался штраф, а с 1928 года в Чикаго существует сеть станций для наблюдения за загрязнением воздуха.

В СССР первые исследования по гигиене атмосферного воздуха проводились в 1929-1930 годах. Их целью было получить научно обоснованные материалы о степени и характере загрязнения атмосферного воздуха в связи с развитием и реконструкцией городов и строительством новых предприятий. Полученные данные использовались при разработке первых элементов санитарного законодательства о размещении промышленных предприятий и принципов организации санитарно-защитных зон между жилыми районами и предприятиями.

Постановление Совета Министров СССР от 29 мая 1949 г. «О мерах борьбы с загрязнением атмосферного воздуха и об улучшении санитарно-гигиенических условий населенных мест» положило начало проведению организованных мероприятий по охране атмосферного воздуха в масштабе всей страны, началось активное развитие законодательства об охране атмосферного воздуха.

На Конференции ООН по проблемам окружающей среды (Стокгольм, 1972 год) в рамках декларации был принят принцип о том, что природные ресурсы Земли, включая воздух, должны быть сохранены на благо нынешнего и будущих поколений путем тщательного планирования и управления по мере необходимости.

Стокгольмская конференция положила начало быстрому развитию национальных и международных организаций, направляющих и координирующих деятельность по сохранению окружающей среды. Если до Стокгольмской конференции государственные органы по охране окружающей среды были только в Швеции, США, Канаде, Великобритании и Японии, то после неё подобные структуры были организованы практически во всех государствах, в том числе в СССР.

В настоящее время на территории Российской Федерации действуют федеральные законы «Об охране окружающей среды» и «Об охране атмосферного воздуха», которые устанавливают правовые основы охраны атмосферного воздуха.

Материал подготовлен на основе информации: http://data.mos.ru/,  http://www.mosecom.ru/, http://www.meteorf.ru/, http://www.mnr.gov.ru/, http://www.ecoethics.ru/, http://energetika.in.ua/.

In [4]:
air = get_dataset('2457')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10013 entries, 0 to 10012
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               10013 non-null  int64 
 1   Date             10013 non-null  object
 2   global_id        10013 non-null  int64 
 3   AdmArea          10013 non-null  object
 4   District         10013 non-null  object
 5   Longitude_WGS84  10013 non-null  object
 6   Latitude_WGS84   10013 non-null  object
 7   Location         10013 non-null  object
 8   Results          10013 non-null  object
 9   geoData          10013 non-null  object
dtypes: int64(2), object(8)
memory usage: 782.4+ KB
None


In [5]:
ecology = fill_ecology(ecology, 'air', 'Results', 'выявлен', 'Date', 'Date', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291 entries, 1 to 9060
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   start   291 non-null    object
 1   end     291 non-null    object
 2   air     291 non-null    object
dtypes: object(3)
memory usage: 9.1+ KB


## Mean

ID:2453

**Среднямесячное загрязнение воздуха опасными веществами** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/2453)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=215804)

Набор данных содержит информацию о средних за месяц концентрациях загрязняющих веществ в атмосферном воздухе Москвы, измеряемых автоматическими станциями контроля загрязнения атмосферы (АСКЗА).

Датасет позволяет получить информацию в разрезе каждой отдельной АСКЗА о средних за месяц концентрациях загрязняющих веществ в абсолютных единицах и в сравнении с предельно допустимыми среднесуточными концентрациями.

Основу мониторинга атмосферного воздуха города Москвы составляет сеть стационарных АСКЗА, которые в круглосуточном непрерывном режиме (с периодичностью один раз в двадцать минут) измеряют содержание в атмосферном воздухе загрязняющих веществ, характерных для выбросов автотранспорта и промышленных предприятий города Москвы.

Сеть автоматических станций мониторинга атмосферного воздуха включает в себя стационарные станции, место размещение которых сохраняется постоянно на протяжении долгого времени, и мобильные станции, размещаемые временно для решения локальных задач (выявление источника загрязнения, определение эффективности природоохранных мероприятий и т.д.).

Датасет включает информацию по стационарным АСКЗА, постоянность размещения которых на одной территории позволяет контролировать динамику содержания загрязняющих веществ в долговременной перспективе.

При ознакомлении с данными датасета важно учитывать, что АСКЗА в системе мониторинга атмосферного воздуха города Москвы ориентированы на контроль загрязнения воздуха на территориях различного функционального назначения: территории вблизи автомагистралей, жилые территории, находящиеся под влиянием смешанных антропогенных источников загрязнений; природные территории, «новая» Москва.

Перечень загрязняющих веществ на АСКЗА определяется функциональным назначением контролируемой территории. Наибольшее количество загрязняющих веществ измеряется вблизи автотрасс и на территориях, находящихся под воздействием очистных сооружений и нефтеперерабатывающего завода.

В общем, список измеряемых на станции загрязняющих веществ может широко варьироваться от 3-х основных параметров (оксид углерода, диоксид и оксид азота) до 10, включая специфические, такие как сероводород (индикатор выбросов нефтехимических производств и эмиссии очистных сооружений канализации).

Среднемесячные концентрации контролируемых параметров представлены как в абсолютных единицах (мг/м3), так и в долях ПДКсс (предельно допустимая среднесуточная концентрация химического вещества в воздухе населённых мест). Концентрация кислорода (О2) выражена в процентной плотности (%).

Благодаря реализуемым природоохранным мероприятиям, долгосрочная динамика демонстрирует постепенное снижение содержания загрязняющих веществ в атмосферном воздухе Москвы, в том числе оксида углерода и оксида азота. В первую очередь, это обусловлено существенным обновлением автопарка города, произошедшим, в том числе, из-за введения ограничения на въезд грузового транспорта и автобусов низких экологических классов, введением требований к качеству моторного топлива и другими крупномасштабными природоохранными мероприятиями, реализуемыми на крупных предприятиях.

Следует отметить, что существенное влияние на уровень загрязнения атмосферного воздуха оказывают метеорологические условия (в том числе слабый ветер, отсутствие осадков), которые приводят к повышению уровня загрязнения атмосферного воздуха. Как правило, такие условия отмечаются преимущественно в ночные и утренние часы.

Сравнительный анализ содержания загрязняющих веществ в атмосферном воздухе Москвы и ряда крупных городов мира (Лондон, Париж, Гонконг, Стокгольм, Нью-Йорк и др.) показывает, что в целом уровень загрязнения воздушного бассейна в городе Москве сравним с уровнем загрязнения в других городах.

Среднегодовые концентрации загрязняющих веществ во всех рассматриваемых городах характеризуются высокой пространственной изменчивостью. Максимальный уровень загрязнения воздуха как в Москве, так и в других крупных городах, отмечается на территориях, прилегающих к крупным автотрассам.

Подробная информация о загрязнении воздуха в Москве представлена  на сайте ГПБУ «Мосэкомониторинг» (www.mosecom.ru). На сайте размещается информация по всем действующим АСКЗА на территории Москвы в режиме реального времени за текущий день, неделю, месяц и год, а также публикуются квартальные бюллетени, ежегодные госдоклады.

Материал подготовлен на основе информации: http://data.mos.ru/, http://www.mosecom.ru,  ГН 2.1.6.1338-03 «Предельно допустимые концентрации (ПДК) загрязняющих веществ в атмосферном воздухе населенных мест», РД 52.04.186-89 «Руководство по контролю загрязнения атмосферы», Федерального закона  № 96-ФЗ «Об охране атмосферного воздуха».

In [6]:
mean = get_dataset('2453')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30381 entries, 0 to 30380
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   global_id                        30381 non-null  int64  
 1   ID                               30381 non-null  int64  
 2   Period                           30381 non-null  object 
 3   StationName                      30381 non-null  object 
 4   SurveillanceZoneCharacteristics  30381 non-null  object 
 5   AdmArea                          30381 non-null  object 
 6   District                         30381 non-null  object 
 7   Location                         30381 non-null  object 
 8   Parameter                        30381 non-null  object 
 9   MonthlyAverage                   30381 non-null  float64
 10  MonthlyAveragePDKss              17761 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 2.5+ MB
None


In [7]:
mean = mean[mean['MonthlyAveragePDKss'].astype('float64') > 1]

In [8]:
%%time
mean = to_geoData(mean, 'Location')

Загружено 31343 записей из cache.gz в кэш
Сохранено 31343 записей в cache.gz
CPU times: user 585 ms, sys: 2.47 ms, total: 587 ms
Wall time: 593 ms


In [9]:
ecology = fill_ecology(ecology, 'mean', 'Period', '', 'Period', 'Period', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1194 entries, 1 to 29637
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   start   1194 non-null   object
 1   end     1194 non-null   object
 2   air     291 non-null    object
 3   mean    903 non-null    object
dtypes: object(4)
memory usage: 46.6+ KB


##  Noise

ID:2449

**Шумовое загрязнение** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/2449)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=203926)

Набор данных позволяет получить информацию о месте проведения измерений уровней шума, источнике шума, дате и основных результатах исследования, о наличии/отсутствии превышений установленных нормативов предельно допустимых уровней шума, а также имеется возможность посмотреть местоположение проведенных измерений на карте города Москвы.

Повышенная шумовая нагрузка характерна для всех мегаполисов мира, и Москва не является исключением.

Основными источниками шумового воздействия на территории Москвы являются автотранспорт, железнодорожный транспорт, авиатранспорт, промышленные предприятия, строительные площадки.

Организацией, осуществляющей проведение государственного экологического мониторинга уровней шума на территории города Москвы, является подведомственное Департаменту природопользования государственное природоохранное бюджетное учреждение «Мосэкомониторинг». На территории города Москвы измерения уровней шума проводятся специализированной акустической службой ГПБУ «Мосэкомониторинг» как в дневное, так и в ночное время суток.

В Российской Федерации установлены санитарные нормы предельно допустимого уровня шума для различных городских территорий. В случае превышения нормативов допустимого уровня шума информация направляется в соответствующие органы исполнительной власти для принятия мер.

Человек всегда жил в мире звуков и шума. Шум является неотъемлемой частью городской жизни, так что каждый житель мегаполиса в той или иной степени подвергается воздействию повышенного шума в течение всего дня.

По данным Всемирной организации здравоохранения, высокие уровни шума могут вызвать такие негативные последствия для здоровья, как повышенная раздражимость, нарушения сна, ухудшение психического здоровья, поведенческие эффекты, снижение работоспособности и даже заболевания сердечно-сосудистой системы. При этом люди наиболее чувствительны к шуму в ночное время.

Однако дискомфорт вызывает не только шумовое загрязнение, но и полное отсутствие шума. Более того, звуки определённой силы повышают работоспособность и стимулируют процесс мышления (в особенности процесс счёта) и, наоборот, при полном отсутствии шумов человек теряет работоспособность и испытывает стресс.

Самое тихое место в мире согласно Книге рекордов Гиннеса находится в Южном Миннеаполисе в Лабораториях Орфилда (штат Миннесота, США), безэховой тестовой камере, блокирующей 99,99% внешних звуков, и никто еще не смог находится в этой комнате более 45 минут.

Первые в мире нормы по ограничению шума  «Временные санитарные нормы и правила по ограничению шума на производстве» были разработаны в СССР и утверждены Министерством здравоохранения СССР в 1956 году в качестве обязательных для всех заводов и проектных организаций. Эти нормативы первоначально разрабатывались для определения условий вредности труда, а впоследствии на основании опыта применения первых норм по ограничению шума, рекомендаций Международной организации по стандартизации и большого числа работ, посвященных изучению различных аспектов влияния шума на человека, Министерством здравоохранения СССР были также утверждены санитарные нормы в отношении территории поселений.

Однако о неблагоприятном воздействии шума на живые организмы было известно еще задолго до появления санитарных норм по шуму и возникновения таких наук, как акустика и аудиология. На Руси, когда стерлядь входила в малые реки на нерест, церкви не звонили в колокола.

В настоящее время на территории Российской Федерации установлены нормативы допустимого уровня шума для различных городских территорий. Так, территории, непосредственно прилегающие к жилым домам, зданиям поликлиник и амбулаторий, домам отдыха и пансионатов, не должны превышать 55 дБА по эквивалентному и 70 дБА по максимальному уровням звука, для ночного времени суток допустимый уровень шума снижается на 10 дБА.

Материал подготовлен на основе информации: 

http://web-local.rudn.ru/, http://www.trudcontrol.ru/, http://mirfactov.com/, http://data.mos.ru.

In [10]:
noise = get_dataset('2449')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13095 entries, 0 to 13094
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               13095 non-null  int64 
 1   Date             13095 non-null  object
 2   global_id        13095 non-null  int64 
 3   AdmArea          13095 non-null  object
 4   District         13095 non-null  object
 5   Longitude_WGS84  13095 non-null  object
 6   Latitude_WGS84   13095 non-null  object
 7   Location         13095 non-null  object
 8   TerritoryType    13095 non-null  object
 9   NoiseCategory    13095 non-null  object
 10  Results          13095 non-null  object
 11  geoData          13095 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.2+ MB
None


In [11]:
ecology = fill_ecology(ecology, 'noise', 'Results', 'выявлен', 'Date', 'Date', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4612 entries, 1 to 13094
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   start   4612 non-null   object
 1   end     4612 non-null   object
 2   air     291 non-null    object
 3   mean    903 non-null    object
 4   noise   3418 non-null   object
dtypes: object(5)
memory usage: 216.2+ KB


## Dirt

ID:2452

**Загрязнения почвы** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/2452)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=218873)

Набор данных содержит результаты ежегодного экологического мониторинга почв в городе Москве за 2015 год.

На Портале размещены данные о качестве почв на 253 площадках постоянного мониторинга, расположенных на территории города Москвы. Для удобства ориентирования предусмотрена возможность просмотра местоположения площадок постоянного мониторинга на карте города Москвы.

Сеть мониторинга за состоянием почв города Москвы включает в себя более 1300 площадок постоянного мониторинга, ежегодно из которых обследуется порядка 200-300. Наблюдательная сеть сформирована с учетом территориального деления (охватывает все административные округа города Москвы) и функционального зонирования (зоны ООПТ, природных и озелененных территорий, жилые, общественные, производственные зоны, а также зоны транспортной инфраструктуры), что позволяет получать максимально полную структурированную информацию о современном состоянии почвенного покрова в городе, отслеживать тенденции изменения состояния почв и выявлять наиболее актуальные проблемы в данной области.

Оценка качества почв города Москвы на основании суммарного показателя загрязнения Zc проводится по содержанию тяжелых металлов: цинка, кадмия, свинца, мышьяка, ртути (вещества I класса опасности), никеля, меди (вещества II класса опасности).

По результатам ежегодного экологического мониторинга 2015 года установлено, что почвы большей части территории города Москвы (более 94 %) характеризуются минимальными уровнями загрязнения (Zc < 16).

Продовольственной и сельскохозяйственной организацией ООН (ФАО) 2015 год был объявлен Международным годом почв.

Генеральный директор ФАО Жозе Грациану да Силва заявил: «Разнообразие ролей, которые играют почвы в нашей жизни, зачастую остается незамеченным. Почвы не умеют говорить, и мало кто выступает в их защиту».

В.И.Вернадский отнес почву к особым биокосным телам природы, которые занимают промежуточное положение между живыми организмами и косными телами (горные породы, минералы).

Почва является вторым после океанов крупнейшим хранилищем углерода на планете (в ней содержится до 75 % углерода). Естественная способность почвы сохранять углерод имеет важное значение для смягчения и адаптации к изменению климата, а также для повышения устойчивости к наводнениям и засухе.

Почва - главный регулятор состава атмосферы Земли. Обусловлено это деятельностью почвенных микроорганизмов, в огромных количествах производящих разнообразные газы - азот и его окислы, кислород, диоксид и оксид углерода, метан и другие углеводороды, сероводород, ряд прочих летучих соединений.

Ученые обнаружили, что почва является одним из наших крупнейших резервуаров биоразнообразия, содержащих почти треть всей жизни планеты! Чайная ложка почвы может быть домом для миллиардов микробов. И жизнь, что рождается в почве, составляет сложную сеть взаимосвязей.

Микроорганизмы, которые живут в почве, используются в медицине для лечения заболеваний. Почти все антибиотики, которые помогают нам бороться с инфекциями, происходят из почвы. Эти организмы дали нам много жизненно необходимых лекарств, таких как антибиотик стрептомицин и препарат циклоспорин, используемый для предотвращения отторжения трансплантатов.

В городах почвы выполняют важные экологические функции: обеспечивают условия для произрастания зеленых насаждений, накапливают в толще загрязняющие вещества и удерживают их от проникновения в грунтовые воды. Биоразнообразие почв исключительно важно для поддержания почвенных функций и, таким образом, экосистемных товаров и услуг, связанных с почвами.

Материалы подготовлены на основе информации: «Методические рекомендации по определению степени загрязнения городских почв и грунтов и проведению инвентаризации территорий, требующих рекультивации» (Москва, 2004), Методические указания МУ 2.1.7.730-99 «Гигиеническая оценка качества почвы населенных мест», СанПиН 4266-87 Методические указания по оценке степени опасности загрязнения почвы химическими веществами, http://ru-ecology.info.

In [12]:
dirt = get_dataset('2452')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2408 entries, 0 to 2407
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2408 non-null   int64  
 1   Year                 2408 non-null   int64  
 2   global_id            2408 non-null   int64  
 3   AdmArea              2408 non-null   object 
 4   District             2408 non-null   object 
 5   Address              2408 non-null   object 
 6   EstimatedindexValue  2408 non-null   float64
 7   PollutionCategory    2408 non-null   object 
 8   PollutionLevel       2408 non-null   object 
 9   geoData              2408 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 188.2+ KB
None


In [13]:
ecology = fill_ecology(ecology, 'dirt', 'PollutionCategory', 'опасная', 'Year', 'Year', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4736 entries, 1 to 2254
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   start   4736 non-null   object
 1   end     4736 non-null   object
 2   air     291 non-null    object
 3   mean    903 non-null    object
 4   noise   3418 non-null   object
 5   dirt    124 non-null    object
dtypes: object(6)
memory usage: 259.0+ KB


##  Digging

ID:62501

**Землянные работы** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/62501)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=218818)

В соответствии с Правилами проведения земляных работ, установки временных ограждений, размещения временных объектов в городе Москве, утвержденных Постановлением Правительства Москвы от 19.05.2015 № 299-ПП.

Данные о действующих зарегистрированных уведомлениях на проведения земляных работ (установки временных ограждений, размещения временных объектов), осуществляемых в целях проведения работ, финансируемых за счет средств бюджета города Москвы или за счет средств Московского фонда реновации жилой застройки, автономной некоммерческой организации "Развитие социальной инфраструктуры", автономной некоммерческой организации "Развитие спортивных и инфраструктурных объектов" включают в себя:

    Номер уведомления
    Дата регистрации
    Наименование организации-балансодержателя коммуникаций
    Наименование организации-производителя работ
    Цель производства работ
    Назначения объекта
    Адрес близлежащего строения
    Округ
    Район
    Плановая дата начала работ
    Плановая дата окончания работ

In [14]:
digging = get_dataset('62501')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6740 entries, 0 to 6739
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   RegistrationNumberNotifications               6740 non-null   object
 1   Date                                          6740 non-null   object
 2   StateCustomer                                 6740 non-null   object
 3   StateCustomerINN                              6740 non-null   object
 4   WorkType                                      6740 non-null   object
 5   EarthworkObjectives                           1015 non-null   object
 6   ObjectivesOfTheInstallationOfTemporaryFences  6685 non-null   object
 7   ObjectivesOfThePlacementOfTemporaryObjects    2301 non-null   object
 8   AdmArea                                       6740 non-null   object
 9   District                                      6740 non-null   object
 10  

In [15]:
ecology = fill_ecology(ecology, 'digging', 'WorkStartDate', '', 'WorkStartDate', 'WorkEndDate', 'geoData', 'Date')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11476 entries, 1 to 6739
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   start    11476 non-null  object
 1   end      11476 non-null  object
 2   air      291 non-null    object
 3   mean     903 non-null    object
 4   noise    3418 non-null   object
 5   dirt     124 non-null    object
 6   digging  6740 non-null   object
dtypes: object(7)
memory usage: 717.2+ KB


## Capital

ID:62963

**Капитальный ремонт жилых помещений** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/62963)

[Датасет CSV](https://data.mos.ru/odata/export/catalog?idFile=218822)

[Датасет JSON](https://data.mos.ru/odata/export/catalog?idFile=220137)

Каталог набора открытых данных «Работы по капитальному ремонту многоквартирных домов» позволяет получить информацию о ходе выполнения работ на объектах городского хозяйства в части мероприятий развития государственных программ города Москвы и обеспечения своевременного и качественного сбора данных по работам, выполняемым на объектах городского хозяйства.

In [16]:
capital = get_dataset('62963')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3490 entries, 0 to 3489
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ObjectAddress      3490 non-null   object
 1   AdmArea            3490 non-null   object
 2   District           3490 non-null   object
 3   WorkEssence        3490 non-null   object
 4   DetailedWork       3490 non-null   object
 5   WorkName           3490 non-null   object
 6   YearOfWork         3490 non-null   int64 
 7   WorkVolume         3490 non-null   object
 8   ContractType       3490 non-null   object
 9   ContractNumber     3490 non-null   object
 10  CustomerName       3490 non-null   object
 11  CustomerID         3490 non-null   object
 12  ContractorName     3490 non-null   object
 13  ContractStartDate  3490 non-null   object
 14  ContractEndDate    3490 non-null   object
 15  ContractStatus     3490 non-null   object
 16  StartDate          3490 non-null   object


In [17]:
%%time
capital = to_geoData(capital, 'ObjectAddress')

Сохранено 31343 записей в cache.gz
CPU times: user 502 ms, sys: 10.8 ms, total: 512 ms
Wall time: 517 ms


In [18]:
ecology = fill_ecology(ecology, 'capital', 'StartDateActual', '', 'StartDateActual', 'EndDateActual', 'geoData', 'StartDate', 'EndDate')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14966 entries, 1 to 3489
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   start    14966 non-null  object
 1   end      14966 non-null  object
 2   air      291 non-null    object
 3   mean     903 non-null    object
 4   noise    3418 non-null   object
 5   dirt     124 non-null    object
 6   digging  6740 non-null   object
 7   capital  3490 non-null   object
dtypes: object(8)
memory usage: 1.0+ MB


## House_lines

ID:62441

**Датасет ремонтных работ ЖКХ подъездных путей жилых домов** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/62441)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=216816)

Набор данных содержит информацию о локальных ремонтных работах на объектах городского хозяйства Москвы, которые проводятся учреждениями подведомственными Департаменту жилищно-коммунального хозяйства города Москвы.За исключением работ, проводимых: в рамках уведомлений Объединения административно-технических инспекций города Москвы (ОАТИ) на аварийно-восстановительный ремонт инженерных коммуникаций, сооружений и дорог; в рамках уведомлений ОАТИ на проведение земляных работ, осуществляемых в целях проведения работ, финансируемых за счет средств бюджета города Москвы; в рамках ордеров ОАТИ на производство земляных работ, установку временных ограждений и размещение временных объектов в пределах установленных границ города Москвы.

Работы по локальному ремонту асфальтобетонного покрытия на улично-дорожной сети города Москвы проводятся в рамках технического содержание объектов дорожного хозяйства (далее также - ОДХ).

- техническое содержание ОДХ - комплекс работ по устранению (ремонту) возникающих локальных повреждений и дефектов дорожных покрытий,.

Материал подготовлен на основании постановления Правительства Москвы от 16 декабря 2014 г. N 762-ПП «Об утверждении требований к санитарно-техническому содержанию объектов дорожного хозяйства улично-дорожной сети города Москвы и порядка выполнения работ по капитальному ремонту, текущему ремонту, разметке и содержанию объектов дорожного хозяйства улично-дорожной сети города Москвы

In [19]:
house_lines = get_dataset('62441')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31582 entries, 0 to 31581
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Works                    31582 non-null  object
 1   Customer                 31582 non-null  object
 2   ActualStartDate          31582 non-null  object
 3   WorkReason               31582 non-null  object
 4   ActualEndDate            31582 non-null  object
 5   AdmArea                  31582 non-null  object
 6   District                 31582 non-null  object
 7   AddressOfNearbyBuilding  31582 non-null  object
 8   WorkPlaceDescription     31582 non-null  object
 9   global_id                31582 non-null  int64 
 10  geoData                  31582 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.7+ MB
None


In [20]:
ecology = fill_ecology(ecology, 'house_lines', 'ActualStartDate', '', 'ActualStartDate', 'ActualEndDate', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46548 entries, 1 to 31581
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   start        46548 non-null  object
 1   end          46548 non-null  object
 2   air          291 non-null    object
 3   mean         903 non-null    object
 4   noise        3418 non-null   object
 5   dirt         124 non-null    object
 6   digging      6740 non-null   object
 7   capital      3490 non-null   object
 8   house_lines  31582 non-null  object
dtypes: object(9)
memory usage: 3.6+ MB


## Clinics

ID:62581

**Ремонт поликлиник** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/62581)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=208952)

Проведение мероприятий по капитальному ремонту объектов здравоохранения в городе Москве осуществляется в рамках Государственной программы города Москвы «Развитие здравоохранения города Москвы (Столичное здравоохранение)».

В план работ на 2021 год включены 44 объекта, площадью более 181 тыс.кв.м. Из них 26 взрослых и 16 детских поликлиник, а также 2 медицинских учреждения смешанного типа.

В рамках капитального ремонта, в соответствии с утвержденным стандартом оснащения поликлиник «Новый московский стандарт поликлиник» выполняется: замена всех инженерных систем, лифтов, окон и дверей, устройство вентиляции и кондиционирования, утепление фасадов и высококачественная отделка помещений, интеграция в системы «ЕМИАС», приспособление помещений для нужд маломобильных граждан, устройство системы доступа.

Кроме того, будут благоустроены территории, прилегающие к этим 44 поликлиникам 2021 года. Пространства около них обустроят в единой стилистике. Общая площадь благоустройства составит более 26 га.

In [21]:
clinics = get_dataset('62581')
# clinics = clinics[clinics['ActualStartDate'].notna()] # Удаление строк с пустыми значениями в столбце ActualStartDate

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ObjectImpName     280 non-null    object
 1   WorkType          280 non-null    object
 2   WorkArea          280 non-null    object
 3   Address           280 non-null    object
 4   AdmArea           280 non-null    object
 5   District          280 non-null    object
 6   WorkYear          280 non-null    int64 
 7   Work              280 non-null    object
 8   ContractNumber    280 non-null    object
 9   Customer          280 non-null    object
 10  Contractor        280 non-null    object
 11  PlannedStartDate  280 non-null    object
 12  PlannedEndDate    280 non-null    object
 13  ActualStartDate   280 non-null    object
 14  ActualEndDate     280 non-null    object
 15  WorkStatus        280 non-null    object
 16  global_id         280 non-null    int64 
 17  geoData         

In [22]:
ecology = fill_ecology(ecology, 'clinics', 'PlannedEndDate', '', 'ActualStartDate', 'ActualEndDate', 'geoData', 'PlannedStartDate', 'PlannedEndDate')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46828 entries, 1 to 279
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   start        46828 non-null  object
 1   end          46828 non-null  object
 2   air          291 non-null    object
 3   mean         903 non-null    object
 4   noise        3418 non-null   object
 5   dirt         124 non-null    object
 6   digging      6740 non-null   object
 7   capital      3490 non-null   object
 8   house_lines  31582 non-null  object
 9   clinics      280 non-null    object
dtypes: object(10)
memory usage: 3.9+ MB


## Roads

ID:62101

**Дорожные работы** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/62101)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=192235)

Объект дорожного хозяйства - искусственное сооружение, предназначенное для безопасного движения транспорта и пешеходов в любое время года независимо от природно-климатических условий.

К объектам дорожного хозяйства относятся городские магистрали, улицы, проезды, площади, переулки и другие объекты, определенные Классификацией объектов дорожного хозяйства, утвержденной постановлением Правительства Москвы от 9 ноября 1999 г. N 1018 "Об утверждении Правил санитарного содержания территорий, организации уборки и обеспечения чистоты и порядка в г. Москве".

Текущий ремонт ОДХ - это комплекс работ по восстановлению транспортно-эксплуатационного состояния отдельных элементов ОДХ до уровня, отвечающего нормам безопасного движения автотранспорта и пешеходов.

Задача текущего ремонта состоит в восстановлении транспортно-эксплуатационного состояния отдельных элементов ОДХ до уровня, позволяющего обеспечить выполнение нормативных требований в период до очередного капитального или текущего ремонта при интенсивности движения, не превышающей расчетную для данного ОДХ.

Критерием для назначения текущего ремонта элементов ОДХ является снижение потребительских свойств ОДХ до предельно допустимых значений, когда их восстановление работами по техническому содержанию невозможно или экономически нецелесообразно.

Текущий ремонт дорожного покрытия картами размером от 100 кв. м и выше:

- восстановление изношенного верхнего слоя дорожной одежды с обеспечением требуемой ровности и шероховатости;

- ликвидация колей и неровностей на проезжей части с заменой нестабильных слоев дорожного покрытия с обязательным фрезерованием на ширину одной или нескольких полос движения или на всю ширину покрытия с укладкой нового слоя из сдвигоустойчивого и износостойкого асфальтобетона;

- при необходимости устройство покрытий из асфальтобетонных смесей на тротуарах с предварительным фрезерованием или разборкой.

Материал подготовлен на основании постановления Правительства Москвы от 16 декабря 2014 г. N 762-ПП «Об утверждении требований к санитарно-техническому содержанию объектов дорожного хозяйства улично-дорожной сети города Москвы и порядка выполнения работ по капитальному ремонту, текущему ремонту, разметке и содержанию объектов дорожного хозяйства улично-дорожной сети города Москвы», а также открытых ресурсов сети Интернет.

In [23]:
roads = get_dataset('62101')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3842 entries, 0 to 3841
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   WorksPlace           3842 non-null   object
 1   WorkYear             3842 non-null   int64 
 2   OnTerritoryOfMoscow  3842 non-null   object
 3   AdmArea              3819 non-null   object
 4   District             3819 non-null   object
 5   WorksBeginDate       3842 non-null   object
 6   PlannedEndDate       3842 non-null   object
 7   ActualBeginDate      3842 non-null   object
 8   ActualEndDate        3842 non-null   object
 9   WorksType            3842 non-null   object
 10  WorksStatus          3842 non-null   object
 11  WorkReason           3842 non-null   object
 12  Customer             3842 non-null   object
 13  Contractor           3842 non-null   object
 14  global_id            3842 non-null   int64 
 15  geoData              3842 non-null   object
dtypes: int

In [24]:
ecology = fill_ecology(ecology, 'roads', 'PlannedEndDate', '', 'ActualBeginDate', 'ActualEndDate', 'geoData', 'WorksBeginDate', 'PlannedEndDate')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50670 entries, 1 to 3841
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   start        50670 non-null  object
 1   end          50670 non-null  object
 2   air          291 non-null    object
 3   mean         903 non-null    object
 4   noise        3418 non-null   object
 5   dirt         124 non-null    object
 6   digging      6740 non-null   object
 7   capital      3490 non-null   object
 8   house_lines  31582 non-null  object
 9   clinics      280 non-null    object
 10  roads        3842 non-null   object
dtypes: object(11)
memory usage: 4.6+ MB


## Parks

ID:61041

**"Благоустройство" парков** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/61041)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=208745)

Проведение мероприятий по благоустройству парков и озелененных территорий в Москве осуществляется в рамках Государственной программы города Москвы «Развитие городской среды», утвержденной постановлением Правительства Москвы № 476-ПП от 7 октября 2011 года (ред. от 10.10.2016 г.).

Благоустройство парковых территорий носит комплексный характер. Как правило, при благоустройстве парковых территорий обеспечивается проведение следующих видов работ:

- озеленение (посадка деревьев и кустарников, разбивка газонов и цветников, создание элементов вертикального и мобильного озеленения);

- обустройство пляжных зон и пристаней;

- создание искусственных водоёмов и фонтанов;

- создание дорожно-тропиночной сети;

- установка малых архитектурных форм (элементы монументально-декоративного оформления, трельяжи, шпалеры, перголы, цветочницы, вазоны, городская мебель: скамейки, столы, садовые диваны, урны, контейнеры для сбора мусора);

- организация зон тихого и активного отдыха, детских и спортивных площадок;

- повсеместное современное функциональное освещение территории - устройство навигационной и информационной инфраструктуры;

- обеспечение территории системами безопасности и видеонаблюдения;

- создание безбарьерной среды для маломобильных групп населения;

- иные виды работ, отвечающие тематике парковой территории.

В зависимости от функционального назначения парковой территории, ее пространственно-площадных характеристик выполняются те или иные виды указанных выше благоустроительных работ.

In [25]:
parks = get_dataset('61041')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      37 non-null     object
 1   WorkType                37 non-null     object
 2   AdmArea                 37 non-null     object
 3   District                37 non-null     object
 4   ObjectCitySignificance  37 non-null     object
 5   Works                   37 non-null     object
 6   WorkScopeModifyReason   37 non-null     object
 7   ContractNumber          37 non-null     object
 8   ContractorName          37 non-null     object
 9   Customer                37 non-null     object
 10  WorkYear                37 non-null     int64 
 11  StartDatePlanned        37 non-null     object
 12  EndDatePlanned          37 non-null     object
 13  StartDateActual         37 non-null     object
 14  EndDateActual           37 non-null     object
 15  WorkEnti

In [26]:
ecology = fill_ecology(ecology, 'parks', 'EndDatePlanned', '', 'StartDateActual', 'EndDateActual', 'geoData', 'StartDatePlanned', 'EndDatePlanned')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50707 entries, 1 to 36
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   start        50707 non-null  object
 1   end          50707 non-null  object
 2   air          291 non-null    object
 3   mean         903 non-null    object
 4   noise        3418 non-null   object
 5   dirt         124 non-null    object
 6   digging      6740 non-null   object
 7   capital      3490 non-null   object
 8   house_lines  31582 non-null  object
 9   clinics      280 non-null    object
 10  roads        3842 non-null   object
 11  parks        37 non-null     object
dtypes: object(12)
memory usage: 5.0+ MB


## Transports

ID:62401

**Строительство транспартной инфраструктуры** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/62401)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=215988)

Для обеспечения пешеходной и транспортной доступности, а также создания качественно новой городской среды, силами Департамента капитального ремонта города Москвы запланированы работы по улучшению пешеходной и транспортной доступности станций Московского метрополитена, станций Московских центральных диаметров, установке шумозащитных экранов, а также локальные мероприятия для обеспечения запуска наземного городского общественного транспорта.
С учетом моделирования транспортной ситуации на расчетный период, определившего существенный рост транспортных и пешеходных потоков, а также увеличения нагрузки на прилегающую к объектам улично-дорожную сеть, предусмотрены мероприятия по расширению и созданию пешеходных зон, приведению к нормативным показателям существующих тротуаров и проезжей части улиц, устройству освещения, оборудованию новых пешеходных переходов, светофорных объектов, установке новых и модернизации существующих остановок общественного транспорта, созданию парковочных мест, озеленению. Выполнение указанных работ позволит создать инфраструктуру, соответствующую современным требованиям как комфорта, так и безопасности.

In [27]:
transports = get_dataset('62401')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   WorkType              108 non-null    object
 1   ObjectImpName         108 non-null    object
 2   WorkAreaBounds        108 non-null    object
 3   AdmArea               108 non-null    object
 4   District              108 non-null    object
 5   Address               108 non-null    object
 6   WorkPlaceDescription  108 non-null    object
 7   WorkYear              108 non-null    int64 
 8   Work                  108 non-null    object
 9   ContractNumber        108 non-null    object
 10  Customer              108 non-null    object
 11  Contractor            108 non-null    object
 12  PlannedStartDate      108 non-null    object
 13  PlannedEndDate        108 non-null    object
 14  ActualStartDate       108 non-null    object
 15  ActualEndDate         108 non-null    ob

In [28]:
ecology = fill_ecology(ecology, 'transports', 'PlannedEndDate', '', 'ActualStartDate', 'ActualEndDate', 'geoData', 'PlannedStartDate', 'PlannedEndDate')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50815 entries, 1 to 107
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   start        50815 non-null  object
 1   end          50815 non-null  object
 2   air          291 non-null    object
 3   mean         903 non-null    object
 4   noise        3418 non-null   object
 5   dirt         124 non-null    object
 6   digging      6740 non-null   object
 7   capital      3490 non-null   object
 8   house_lines  31582 non-null  object
 9   clinics      280 non-null    object
 10  roads        3842 non-null   object
 11  parks        37 non-null     object
 12  transports   108 non-null    object
dtypes: object(13)
memory usage: 5.4+ MB


## City_places 

**"Благоустройство городских пространств"** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

ID: 63201

[Источник](https://data.mos.ru/opendata/63201)

[Датасет CSV](https://data.mos.ru/odata/export/catalog?idFile=218594)

In [29]:
city_places = get_dataset('63201')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3093 entries, 0 to 3092
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ObjectCategory     3093 non-null   object
 1   ObjectAddress      3093 non-null   object
 2   AdmArea            3093 non-null   object
 3   District           3093 non-null   object
 4   WorkName           3093 non-null   object
 5   WorkEssence        3093 non-null   object
 6   DetailedWork       3093 non-null   object
 7   YearOfWork         3093 non-null   int64 
 8   WorkVolume         3093 non-null   object
 9   ContractType       3093 non-null   object
 10  ContractNumber     3093 non-null   object
 11  CustomerName       3093 non-null   object
 12  CustomerID         3093 non-null   object
 13  ContractorName     3093 non-null   object
 14  ContractStartDate  3093 non-null   object
 15  ContractEndDate    3093 non-null   object
 16  ContractStatus     3093 non-null   object


In [30]:
%%time
city_places = to_geoData(city_places, 'ObjectAddress')

Сохранено 31343 записей в cache.gz
CPU times: user 619 ms, sys: 6.83 ms, total: 626 ms
Wall time: 631 ms


In [31]:
ecology = fill_ecology(ecology, 'city_places', 'StartDateActual', '', 'StartDateActual', 'EndDateActual', 'geoData', 'StartDate', 'EndDate')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53908 entries, 1 to 3092
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   start        53908 non-null  object
 1   end          53908 non-null  object
 2   air          291 non-null    object
 3   mean         903 non-null    object
 4   noise        3418 non-null   object
 5   dirt         124 non-null    object
 6   digging      6740 non-null   object
 7   capital      3490 non-null   object
 8   house_lines  31582 non-null  object
 9   clinics      280 non-null    object
 10  roads        3842 non-null   object
 11  parks        37 non-null     object
 12  transports   108 non-null    object
 13  city_places  3093 non-null   object
dtypes: object(14)
memory usage: 6.2+ MB


## Live_area

ID:62961

**"Благоустройство" жилой застройки** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

Бордюры, газоны и т.д.

[Источник](https://data.mos.ru/opendata/62961)

[Датасет CSV](https://data.mos.ru/odata/export/catalog?idFile=218826)

[Датасет JSON](https://data.mos.ru/odata/export/catalog?idFile=220546)

In [32]:
live_area = get_dataset('62961')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127492 entries, 0 to 127491
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ObjectAddress      127492 non-null  object
 1   AdmArea            127492 non-null  object
 2   District           127492 non-null  object
 3   YardID             127492 non-null  int64 
 4   WorkEssence        127492 non-null  object
 5   DetailedWork       127492 non-null  object
 6   WorkName           127492 non-null  object
 7   YearOfWork         127492 non-null  int64 
 8   WorkVolume         127492 non-null  object
 9   ContractType       127492 non-null  object
 10  ContractNumber     127492 non-null  object
 11  CustomerName       127492 non-null  object
 12  CustomerID         127492 non-null  object
 13  ContractorName     127492 non-null  object
 14  ContractStartDate  127492 non-null  object
 15  ContractEndDate    127492 non-null  object
 16  ContractStatus     1

In [33]:
live_area = live_area[live_area['YearOfWork'] > 2016].copy() # Удаляем строки с пустым полем ActualStartDate

In [34]:
%%time
live_area = to_geoData(live_area, 'ObjectAddress')

Сохранено 31343 записей в cache.gz
CPU times: user 1.67 s, sys: 25.2 ms, total: 1.7 s
Wall time: 1.71 s


In [35]:
ecology = fill_ecology(ecology, 'live_area', 'StartDateActual', '', 'StartDateActual', 'EndDateActual', 'geoData', 'StartDate', 'EndDate')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157016 entries, 1 to 127491
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   start        157016 non-null  object
 1   end          157016 non-null  object
 2   air          291 non-null     object
 3   mean         903 non-null     object
 4   noise        3418 non-null    object
 5   dirt         124 non-null     object
 6   digging      6740 non-null    object
 7   capital      3490 non-null    object
 8   house_lines  31582 non-null   object
 9   clinics      280 non-null     object
 10  roads        3842 non-null    object
 11  parks        37 non-null      object
 12  transports   108 non-null     object
 13  city_places  3093 non-null    object
 14  live_area    103108 non-null  object
dtypes: object(15)
memory usage: 19.2+ MB


## Streets

ID:62221

**Благоустройство улиц** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/62221)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=200723)

Проведение мероприятий по комлексному благоустройству улиц и общественных пространств в городе Москве осуществляется в рамках Государственной программы города Москвы «Развитие городской среды», утвержденной постановлением Правительства Москвы № 476-ПП от 7 октября 2011 года (ред. от 10.10.2016 г.).

Департаментом капитального ремонта города Москвы в рамках реализации данной программы в 2020-2023 годах запланировано проведение масштабных работ по комплексному благоустройству улиц, общественных пространств, природных и озеленённых территорий.

В план работ включены более 200 объектов, площадью более 4 800 га, среди них: благоустройство озеленённых территорий, природных комплексов и особо охраняемых природных территорий, благоустройство вылетных магистралей, улиц, общественных пространств и прочих объектов.

На всех реализуемых объектах запланированы мероприятия по комплексному благоустройству с созданием современных и технологичных общественных пространств, включающие в себя работы по: планировке территории, устройству покрытий дорожно-тропиночной сети, освещения и инженерных систем, установке малых архитектурных форм и оборудования, установке нестационарных объектов и прочие, направленных на социально – экономическое развитие города и создание комфортных условий для жителей.

In [36]:
streets = get_dataset('62221')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   WorkType                 24 non-null     object
 1   StreetName               24 non-null     object
 2   ObjectImpName            24 non-null     object
 3   WorkAreaBounds           24 non-null     object
 4   UNOM                     0 non-null      object
 5   Address                  24 non-null     object
 6   Work                     24 non-null     object
 7   RegistryContractNumber   24 non-null     object
 8   EmergencyContractNumber  24 non-null     object
 9   ContractStatus           24 non-null     object
 10  Customer                 24 non-null     object
 11  Contractor               24 non-null     object
 12  WorkYear                 24 non-null     int64 
 13  PlannedStartDate         24 non-null     object
 14  PlannedEndDate           24 non-null     obj

In [37]:
ecology = fill_ecology(ecology, 'streets', 'PlannedEndDate', '', 'ActualStartDate', 'ActualEndDate', 'geoData', 'PlannedStartDate', 'PlannedEndDate')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157040 entries, 1 to 23
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   start        157040 non-null  object
 1   end          157040 non-null  object
 2   air          291 non-null     object
 3   mean         903 non-null     object
 4   noise        3418 non-null    object
 5   dirt         124 non-null     object
 6   digging      6740 non-null    object
 7   capital      3490 non-null    object
 8   house_lines  31582 non-null   object
 9   clinics      280 non-null     object
 10  roads        3842 non-null    object
 11  parks        37 non-null      object
 12  transports   108 non-null     object
 13  city_places  3093 non-null    object
 14  live_area    103108 non-null  object
 15  streets      24 non-null      object
dtypes: object(16)
memory usage: 20.4+ MB


## Trashes

ID:2542

**Помойки (мусорные контейнеры, баки и т.д.)** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/2542)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=23148)

Каталог содержит подробную информацию о местах сбора отходов (контейнерные площадки, площадки для выкатных контейнеров), их адреса, район, административный округ, географические координаты, № площадки.
На территории Российской Федерации раздельный сбор отходов регламентируется положениями Федерального закона от 24.06.1998 № 89-ФЗ и законодательными актами субъектов РФ.
В городе Москве на  основании  постановлений  Правительства  Москвы  от  13.09.2012 № 485-ПП, от 02.10.2013 № 662-ПП и от 10.11.2014 № 644-ПП в Центральном, Северном, Северо-Восточном, Восточном, Юго-Восточном, Юго-Западном, Западном, Северо-Западном и Зеленоградском административных округах города Москвы реализуется новый комплексный метод обращения с отходами, для реализации которого на конкурсной основе заключено девять долгосрочных (на 15 лет) государственных контрактов на оказание услуг по обращению с твёрдыми бытовыми отходами и крупногабаритным мусором, образующимися в многоквартирных домах (далее - Контракт), расположенных на территориях указанных административных округов города Москвы.
Исполнителями Государственных контрактов являются ООО «МКМ-Логистика» (ЗАО, ЮЗАО), ООО «ЭкоЛайн» (ЦАО, САО), ООО «Хартия» (ВАО, СВАО), ООО «МСК-НТ» (ЮВАО, ЗелАО) и ООО «Спецтранс» (СЗАО).
По состоянию на 31.05.2017 в рамках Государственных контрактов в девяти округах создано 15307 мест сбора отходов (далее – МСО), в том числе 13771  для твердых бытовых отходов и 1536 для крупногабаритного мусора. В общей сложности на местах сбора отходов установлено 36687 контейнеров.
В ЦАО в рамках Государственных контрактов создано 1293 МСО, в том числе 1285  ТБО и 8 КГМ.
В САО в рамках Государственных контрактов создано 1806 МСО, в том числе 1627  ТБО, и 179 КГМ.
В СВАО в рамках Государственных контрактов создано 2481 МСО, в том числе 2075  ТБО, и 406 КГМ.
В ВАО в рамках Государственных контрактов создано 2759 МСО, в том числе 2465  ТБО, и 294 КГМ.
В ЮВАО в рамках Государственных контрактов  создано 2115 МСО, в том числе 1741  ТБО, и 374 КГМ.
В ЗелАО в рамках Государственных контрактов создано 383 МСО, в том числе 292  ТБО, и 91 КГМ.
В ЮЗАО в рамках Государственных контрактов  создано 2008 МСО, в том числе 1942  ТБО, и 66 КГМ.
В ЗАО в рамках Государственных контрактов создано 1489 МСО, в том числе 1405  ТБО, и 84 КГМ.
В СЗАО в рамках Государственных контрактов создано 973 МСО, в том числе 939  ТБО, и 34 КГМ.
А знаете ли Вы, что? Кроме того, по ряду МСО осуществляется сбор следующих вторичных материальных ресурсов: макулатуры, стеклянной тары, ПЭТФ-тары, алюминиевой тары в отдельные контейнеры по видам отходов.

Материал подготовлен на основе информации data.mos.ru

In [38]:
trashes = get_dataset('2542')
trashes['start'] = date(day=1, month=1, year=2017)
trashes['end'] = date.today().strftime('%d.%m.%Y')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26820 entries, 0 to 26819
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   global_id          26820 non-null  int64  
 1   Code               26820 non-null  int64  
 2   YardName           26820 non-null  object 
 3   YardType           26820 non-null  object 
 4   YardAddress        26820 non-null  object 
 5   YardArea           16050 non-null  float64
 6   Coordinates        26820 non-null  object 
 7   EntryState         26820 non-null  object 
 8   EntryAddReason     26820 non-null  object 
 9   EntryModifyReason  26820 non-null  object 
 10  EntryDeleteReason  0 non-null      object 
 11  geoData            26820 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 2.5+ MB
None


In [39]:
ecology = fill_ecology(ecology, 'trashes', 'YardName', '', 'start', 'end', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183860 entries, 1 to 26819
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   start        183860 non-null  object
 1   end          183860 non-null  object
 2   air          291 non-null     object
 3   mean         903 non-null     object
 4   noise        3418 non-null    object
 5   dirt         124 non-null     object
 6   digging      6740 non-null    object
 7   capital      3490 non-null    object
 8   house_lines  31582 non-null   object
 9   clinics      280 non-null     object
 10  roads        3842 non-null    object
 11  parks        37 non-null      object
 12  transports   108 non-null     object
 13  city_places  3093 non-null    object
 14  live_area    103108 non-null  object
 15  streets      24 non-null      object
 16  trashes      26820 non-null   object
dtypes: object(17)
memory usage: 25.2+ MB


## Factories

ID:2601

**Промышленные предприятия** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/2601)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=215363)

Набор данных  «Данные по промышленным предприятиям» содержит подробную информацию об основных крупных и средних предприятиях города Москвы в сфере обрабатывающей промышленности, имеющих производственную локализацию на территории города Москвы. 
Промышленные предприятия являются важнейшим источником пополнения городского бюджета налоговыми отчислениями, создают новые высокопроизводительные рабочие места, обеспечивают приток инвестиций в создание конкурентоспособной промышленной продукции.
Закон г. Москвы от 07.10.2015 N 55 «О промышленной политике города Москвы» определяет цели и задачи промышленной политики города Москвы, устанавливает правовые и экономические основы создания условий для эффективного развития промышленности на территории города Москвы, регулирует отношения, возникающие между субъектами деятельности в сфере промышленности, органами государственной власти города Москвы при формировании и реализации промышленной политики.
 В целях расширения и упорядочения мер государственной поддержки в отношении новых инвестиционных проектов и действующих промышленных предприятий принят Закон города Москвы от 07.10.2015 № 54 «Об инвестиционной политике города Москвы и государственной поддержке субъектов инвестиционной деятельности».
Основные мероприятия по поддержке промышленных предприятий города реализуются в рамках подпрограммы «Москва - город для бизнеса и инноваций» Государственной программы «Экономическое развитие и инвестиционная привлекательность города Москвы» (Постановление Правительства Москвы от 11.10.2011 N 477-ПП).
Современная ситуация в России
На обрабатывающие отрасли промышленности приходится более 13% валового регионального продукта Российской Федерации и более 14% от общего числа занятых в экономике страны.
Одним из основных инструментов реализации государственной политики в промышленности являются государственные программы Российской Федерации, ответственным исполнителем которых выступает Минпромторг России: «Развитие промышленности и повышение ее конкурентоспособности», «Развитие авиационной промышленности», «Развитие электронной и радиоэлектронной промышленности» и другие.
В 2014 году был создан Фонд развития промышленности. Фонд предлагает льготные условия софинансирования проектов, направленных на разработку новой высокотехнологичной продукции, техническое перевооружение и создание конкурентоспособных производств на базе наилучших доступных технологий. Для реализации новых промышленных проектов Фонд на конкурсной основе предоставляет целевые займы по ставке 5% годовых сроком до 7 лет в объеме от 50 до 500 млн рублей, стимулируя приток прямых инвестиций в реальный сектор экономики.
Современная ситуация в Москве
Промышленность Москвы представлена более 20 гражданскими отраслями, ведущими из которых являются пищевая, фармацевтическая, электронная, автомобильная, машиностроительная и другие отрасли.
Обрабатывающие отрасли промышленности играют важную роль в экономике города, обеспечивая более 12% в структуре валового регионального продукта, 7% налоговых поступлений в бюджет города Москвы и 7% от общего числа занятых в экономике города.
Для действующих промышленных предприятий города Москвы Правительством Москвы предоставляются следующие меры поддержки:
1. Присвоение статуса промышленного комплекса.
Порядок получения статуса регламентирован постановлением Правительства Москвы от 11.02.2016 № 38-ПП  «О мерах по реализации промышленной и инвестиционной политики в городе Москве». Присвоение статуса позволяет получать предприятиям региональные налоговые льготы по налогу на прибыль, налогу на имущество, земельному налогу и льготы по аренде земли. Критерии, в соответствии с которыми присваиваются статусы, связаны с эффективностью использования земельных участков, производственных площадей, экономической деятельностью промышленного комплекса, в том числе уровнем заработной платы сотрудников промышленных предприятий.
2. Предоставление субсидий на поддержку организаций, осуществляющих производственную деятельность.
В рамках постановления Правительства Москвы от 20.04.2012 № 152-ПП «Об утверждении порядков предоставления субсидий из бюджета города Москвы организациям, осуществляющим на территории города Москвы деятельность в сфере промышленности» промышленным предприятиям предоставляются субсидии по двум направлениям:
- возмещение части затрат на уплату процентов по кредитам, полученным на приобретение отдельных видов производственного оборудования;
- возмещение части затрат на уплату лизинговых платежей по договорам финансовой аренды (лизинга) отдельных видов производственного оборудования.
 А знаете ли вы что?
Снижение налоговой нагрузки для предприятий, получивших статус промышленного комплекса, может составлять до 17%. Подробную информацию о мерах поддержки промышленности Москвы можно узнать на сайте Департамента науки, промышленной политики и предпринимательства города Москвы в разделе «Промышленность Москвы». (http://dnpp.mos.ru/).
Материал подготовлен на основе информации http://dnpp.mos.ru, http://frprf.ru, http://minpromtorg.gov.ru, http://www.consultant.ru, http://data.mos.ru, http://moscow.gks.ru/, http://www.gks.ru/.

In [40]:
factories = get_dataset('2601')
factories['start'] = date(day=1, month=1, year=2017)
factories['end'] = date.today().strftime('%d.%m.%Y')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 840 non-null    int64 
 1   global_id          840 non-null    int64 
 2   FullName           840 non-null    object
 3   ShortName          840 non-null    object
 4   INN                840 non-null    object
 5   OKPO               840 non-null    object
 6   AdmArea            840 non-null    object
 7   District           840 non-null    object
 8   Address            840 non-null    object
 9   Category           840 non-null    object
 10  Specialization     840 non-null    object
 11  OKVED              840 non-null    object
 12  OKVED_Description  840 non-null    object
 13  geoData            840 non-null    object
dtypes: int64(2), object(12)
memory usage: 92.0+ KB
None


In [41]:
ecology = fill_ecology(ecology, 'factories', 'Category', '', 'start', 'end', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184700 entries, 1 to 839
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   start        184700 non-null  object
 1   end          184700 non-null  object
 2   air          291 non-null     object
 3   mean         903 non-null     object
 4   noise        3418 non-null    object
 5   dirt         124 non-null     object
 6   digging      6740 non-null    object
 7   capital      3490 non-null    object
 8   house_lines  31582 non-null   object
 9   clinics      280 non-null     object
 10  roads        3842 non-null    object
 11  parks        37 non-null      object
 12  transports   108 non-null     object
 13  city_places  3093 non-null    object
 14  live_area    103108 non-null  object
 15  streets      24 non-null      object
 16  trashes      26820 non-null   object
 17  factories    840 non-null     object
dtypes: object(18)
memory usage: 26.8+ MB


## Markets

ID:654

**Рынки, ярмарки** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/654)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=206400)

Каталог содержит информацию о розничных рынках, организованных на территории города Москвы с учетом их типологии (27 розничных рынков, из которых: 22 специализированных сельскохозяйственных рынка; 4 универсальных рынка и 1 радиорынок).

В каталогах отражена информация о фактическом месте расположения рынка с указание административного округа, района, адреса размещения, юридический адресе Управляющей рынком компании, ее наименование и специализация рынка.

Сеть розничных рынков города Москвы является важным механизмом для реализации поставки продовольствия, позволяющая потребителям получать производимую сельскохозяйственную продукцию непосредственно от сельхозпроизводителей минуя звенья посредников.

Существующая сеть из 27 розничных рынков, на которых в настоящий момент предусмотрено более 14 тысяч торговых мест, играет важную роль в обеспечении продовольственными товарами значительного количества предприятий розничной торговли и общественного питания, в том числе объектов мелкорозничной торговли, одновременно являясь поставщиком качественных сельскохозяйственных продуктов.

Развитие форматов торговой деятельности в Москве, который мы видим в последние годы существенно затрагивает и розничные рынки.

Если в период до начала 2000-х годов  задачей деятельности розничных рынков можно было считать преодоление товарного дефицита, то в настоящее время розничные рынки эволюционируют в современные инфраструктурные объекты, удобные и безопасные для жителей.

В настоящее время розничные рынки эволюционируют в современные инфраструктурные объекты, удобные и безопасные для жителей.

В связи с вступлением в силу норм Федерального закона от 30.12.2006 № 271-ФЗ «О розничных рынках и о внесении изменений в Трудовой кодекс Российской Федерации», с 01.01.2016 г. торговая деятельность вне капитальных зданий прекращена. Правительство Москвы реализует программу модернизации традиционных московских рынков, для обеспечения москвичей качественной сельскохяйственной продукцией.

Следуя требованиям законодательства, управляющие рынками компании ведут работы по проектированию и строительству капитальных торговых объектов, которые будут построены на месте некапитальных рынков. В этих торговых объектах также будут реализовываться сельскохозяйственные продукты, и покупатели вновь смогут приобретать полюбившиеся им товары, но уже в удобных,  комфортных условиях, в объектах, отвечающих санитарно-техническим нормам и требованиям безопасности. В Зеленограде на месте одноэтажного контейнерного рынка, расположенного на Крюковской площади, уже пострен новый совеременнный многоэтажный сельскохозяйственный рынок, а в ближайшее вемя управляющей компанией на смежных участках будут построены торговый центр и здание ресторана.

Кроме того, капитальные сельскохозяйственные рынки следуя тенденциям торговли также эволюционируют и становятся современнее и удобнее. Так например, проведены работы по реконструкции, Велозаводского и Москворецкого рынков.

А знаете ли вы что?

В Москве существуют рынки со сложившимся историческим обликом, являющимся неотъемлемой частью архитектурного ансамбля города, при их модернизации собственники стараются сохранить внешний облик наиболее значимых объектов.

Например, на одном из старейших и известнейших рынков города  Москвы, Даниловском рынке, проведены масштабные работы по его реконструкции.

При подготовке материалов использован сайт: mos.ru

In [42]:
markets = get_dataset('654')
markets['start'] = date(day=1, month=1, year=2017)
markets['end'] = date.today().strftime('%d.%m.%Y')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   ID                               26 non-null     int64 
 1   Name                             26 non-null     object
 2   global_id                        26 non-null     int64 
 3   AdmArea                          26 non-null     object
 4   District                         26 non-null     object
 5   Address                          26 non-null     object
 6   ManagementCompany                26 non-null     object
 7   INN                              26 non-null     object
 8   KPP                              26 non-null     object
 9   OGRN                             26 non-null     object
 10  LegalAddressOfManagementCompany  26 non-null     object
 11  MarketType                       26 non-null     object
 12  ExtraInfo                        26 no

In [43]:
ecology = fill_ecology(ecology, 'markets', 'MarketType', '', 'start', 'end', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184726 entries, 1 to 25
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   start        184726 non-null  object
 1   end          184726 non-null  object
 2   air          291 non-null     object
 3   mean         903 non-null     object
 4   noise        3418 non-null    object
 5   dirt         124 non-null     object
 6   digging      6740 non-null    object
 7   capital      3490 non-null    object
 8   house_lines  31582 non-null   object
 9   clinics      280 non-null     object
 10  roads        3842 non-null    object
 11  parks        37 non-null      object
 12  transports   108 non-null     object
 13  city_places  3093 non-null    object
 14  live_area    103108 non-null  object
 15  streets      24 non-null      object
 16  trashes      26820 non-null   object
 17  factories    840 non-null     object
 18  markets      26 non-null      object
dtypes: object(1

## Autostations

ID:1881

**Автовокзалы и автостанции** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/1881)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=203878)

Набор данных содержит подробную информацию об остановочных пунктах, которые используются для обслуживания межрегиональных и международных автобусных маршрутов.

Автовокзалы и автостанции открыты с целью упорядочить движение междугородного транспорта по городу для удобства москвичей и гостей столицы, обеспечения безопасности перевозок, повышения культуры и качества обслуживания пассажиров, создания цивилизованного рынка транспортных услуг.

Наибольшее число автовокзалов и автостанций сконцентрировано в Южном административном округе:

Работа московских автостанций и автовокзалов регламентирована постановлением Правительства города Москвы от 17.12.2015 N 895-ПП «Об установлении в границах города Москвы остановочных пунктов, которые разрешается использовать в качестве начальных и (или) конечных остановочных пунктов по межрегиональным маршрутам регулярных перевозок, о внесении изменения в постановление Правительства Москвы от 15 февраля 2011 г. N 32-ПП и признании утратившими силу правовых актов города Москвы»

Московские автостанции оборудованы перронами для посадки и высадки пассажиров, кассами по продаже билетов, залами ожидания рейса, местами кратковременной стоянки автобусов. Водители проходят предрейсовый медицинский осмотр, автобусы – технический. Пассажирам предоставляется визуальное и звуковое информационное обеспечение, схемы автобусных маршрутов, расписание отправления и прибытия, таблицы стоимости проезда и провоза багажа.

Современная ситуация в России

Автобусный транспорт широко востребован населением России, он перевозит более 11,5 млрд человек ежегодно, пассажирооборот более 120 млрд. пассажиро-километров.

Доля автобусного транспорта в структуре междугородных перевозок составляет более 30 процентов, уступая только железнодорожному. В России действует более 3000 междугородных маршрутов, выполняется более 3 млн. рейсов ежегодно.

Основная проблема рынка: существенная часть междугородных автобусных перевозок выполняется под видом заказных, где требования транспортной безопасности зачастую не выполняются, перевозчики меняют маршруты и время отправления, а также транспортные средства в пути следования, что справедливо вызывает нарекания пассажиров.

Современная ситуация в Москве

Для удобства москвичей и гостей столицы, обеспечения безопасности перевозок, повышения культуры и качества обслуживания пассажиров, создания цивилизованного рынка транспортных услуг в Москве организованы автовокзалы и автостанции для регулярных межрегиональных и международных автобусных маршрутов.

Большей частью автостанций и автовокзалов управляет государственное унитарное предприятие города Москвы «Мосгортранс». Расположение автостанций и автовокзалов на конечных станциях городского наземного транспорта, вблизи вылетных автомагистралей способствует решению проблемы загруженности центральной части города автобусами, прибывающими в столицу из регионов России и зарубежных стран. Близость к станциям метро делает автостанции удобными для прибытия и отправления пассажиров. А названия, соответствующие ближайшим станциям метрополитена, позволяют легко их найти.

А знаете ли вы что?

Приобрести билет на междугородные и международные автобусы с отправлением из Москвы теперь можно не только в кассах автовокзалов и автостанций, но и в более чем 530 терминалах самообслуживания в Москве и Московской области, адреса расположения терминалов на сайте www.av.mos.ru

In [44]:
autostations = get_dataset('1881')
autostations['start'] = date(day=1, month=1, year=2017)
autostations['end'] = date.today().strftime('%d.%m.%Y')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    8 non-null      int64 
 1   Name                  8 non-null      object
 2   global_id             8 non-null      int64 
 3   AdmArea               8 non-null      object
 4   District              8 non-null      object
 5   Address               8 non-null      object
 6   MetroStation          8 non-null      object
 7   MetroLine             8 non-null      object
 8   WiFiAvailability      8 non-null      object
 9   BabyCareRoom          8 non-null      object
 10  SanitationFacilities  8 non-null      object
 11  PublicPhone           8 non-null      object
 12  WebSite               8 non-null      object
 13  WorkingHours          8 non-null      object
 14  BusDestination        8 non-null      object
 15  LocationComment       8 non-null      object

In [45]:
ecology = fill_ecology(ecology, 'autostations', 'Name', '', 'start', 'end', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184734 entries, 1 to 7
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   start         184734 non-null  object
 1   end           184734 non-null  object
 2   air           291 non-null     object
 3   mean          903 non-null     object
 4   noise         3418 non-null    object
 5   dirt          124 non-null     object
 6   digging       6740 non-null    object
 7   capital       3490 non-null    object
 8   house_lines   31582 non-null   object
 9   clinics       280 non-null     object
 10  roads         3842 non-null    object
 11  parks         37 non-null      object
 12  transports    108 non-null     object
 13  city_places   3093 non-null    object
 14  live_area     103108 non-null  object
 15  streets       24 non-null      object
 16  trashes       26820 non-null   object
 17  factories     840 non-null     object
 18  markets       26 non-null      obj

## Gasstations

ID:753

**Aвтозапровки** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/753)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=215812)

Набор данных содержит информацию о результатах контрольных мероприятий, проведённых Департаментом природопользования и охраны окружающей среды города Москвы в отношении автозаправочных станций с последующими лабораторными испытаниями отобранных на АЗС проб моторного топлива.

Экологические требования к качеству моторного топлива на территории города Москвы установлены постановлением Правительства Москвы от 28.12.2004 № 952-ПП «Об утверждении экологических требований к качеству моторного топлива при его обороте в городе Москве», также данным документом установлен перечень контролируемых Департаментом показателей моторного топлива (Приложение 3 к постановлению Правительства Москвы от 28.12.2004 № 952-ПП).

Перечень основных показателей моторного топлива, контролируемых Департаментом:


1. Автомобильные бензины:

– октановое число по исследовательскому методу;

– октановое число по моторному методу;

– концентрация свинца;

– концентрация серы;

– объёмная доля бензола;

- объёмная доля ароматических углеводородов.

2. Топливо дизельное:

– массовая доля полициклических ароматических углеводородов;

– концентрация серы;

- смазывающая способность;

- общее загрязнение.

Дата-сет позволяет получить подробную информацию об объектах реализации моторного топлива (АЗС), посмотреть их местоположение на карте, узнать  дату и результаты контрольных мероприятий. По каждой проверенной Департаментом АЗС можно посмотреть полное наименование юридического лица, осуществляющего деятельность по реализации моторного топлива на момент контрольных мероприятий, точный адрес АЗС.

Современная ситуация в России

Решением Комиссии Таможенного союза от 2 декабря 2015 года № 84 о внесении изменений в технический регламент таможенного союза «О требованиях к автомобильному и авиационному бензину, дизельному и судовому топливу, топливу для реактивных двигателей и мазуту» (ТР ТС 013/2011, на территории Российской Федерации с 1 января 2016 года запрещен выпуск в обращение и обращение дизельного топлива экологического класса Евро-4, а с 1 июля 2016 года выпуск в обращение и обращение автомобильного бензина экологического класса Евро-4.

Постановлением Правительства Российской Федерации от 29 декабря 2012 года        № 1474 «О применении некоторых актов Правительства Российской Федерации по вопросам технического регулирования и об органах государственного контроля (надзора) за соблюдением требований технического регламента таможенного союза «О требованиях к автомобильному и авиационному бензину, дизельному и судовому топливу, топливу для реактивных двигателей и топочному мазуту» установлено, что: Федеральное агентство по техническому регулированию и метрологии осуществляет государственный контроль (надзор) за соблюдением требований к автомобильному бензину, дизельному топливу, судовому топливу и мазуту, установленных Техническим регламентом Таможенного союза.

Современная ситуация по реализации моторного топлива в городе Москве

По итогам 2016 года в результате контрольных мероприятий на АЗС города Москвы выявлено 11 проб (11,2% от общего количества отобранных проб) моторного топлива, не соответствует утвержденным экологическим требованиям, в том числе  8 проб автомобильного бензина и 3 пробы дизельного топлива.

По данным фактам Департаментом виновные лица привлечены к административной ответственности в виде административных штрафов, также виновным лицам выданы предписания на устранение нарушений. В случае невыполнения выданных предписаний виновные лица также привлекаются Департаментом к административной ответственности в виде административных штрафов.

- АЗС на территории города Москвы контролируются Департаментом на предмет соответствия реализуемого моторного топлива экологическим требованиям с 2004 года в границах города до 1 июля 2012 года и с 1 июля 2012 года в современных границах города Москвы.

- В случае выявления фактов реализации на АЗС моторного топлива, не соответствующего экологическим требованиям, установленным на территории города Москвы, Вы вправе обратиться в Единую справочную службу Мэрии Москвы по тел.    8 (495) 777-77-77 либо письменно в Департамент (119019, Москва, ул. Новый Арбат,  д. 11, корп. 1 либо через портал Департамента (http://eco.mos.ru/) с просьбой провести контрольные мероприятия в отношении указанных АЗС. При этом в обращении требуется указать причины обращения (перечислить признаки реализации моторного топлива, не соответствующего установленным экологическим требованиям), точный адрес расположения АЗС на территории города Москвы, Фамилию, Имя, Отчество и контактные данные (адрес заявителя и телефонный номер).

- В соответствии с положениями статьи 4.10 (Производство, оборот нефтепродуктов, не соответствующих экологическим требованиям) Кодекса города Москвы об административных правонарушениях установлена административная ответственность в виде административного штрафа на лиц, осуществляющих предпринимательскую деятельность без образования юридического лица, в размере 50 тысяч рублей; на юридических лиц – 400 тысяч рублей. При этом повторное совершение указанного административного правонарушения влечет наложение административного штрафа на юридических лиц в размере одного миллиона рублей.

Материал подготовлен на основе информации, полученной в результате контрольных мероприятий, проведённых Департаментом, и нормативных правовых актов Российской Федерации и города Москвы, http://data.mos.ru/.

In [46]:
gasstations = get_dataset('753')
gasstations['start'] = date(day=1, month=1, year=2017)
gasstations['end'] = date.today().strftime('%d.%m.%Y')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         308 non-null    int64 
 1   FullName   308 non-null    object
 2   global_id  308 non-null    int64 
 3   ShortName  308 non-null    object
 4   AdmArea    308 non-null    object
 5   District   308 non-null    object
 6   Address    308 non-null    object
 7   Owner      308 non-null    object
 8   TestDate   308 non-null    object
 9   geoData    308 non-null    object
dtypes: int64(2), object(8)
memory usage: 24.2+ KB
None


In [47]:
ecology = fill_ecology(ecology, 'gasstations', 'Owner', '', 'start', 'end', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185042 entries, 1 to 307
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   start         185042 non-null  object
 1   end           185042 non-null  object
 2   air           291 non-null     object
 3   mean          903 non-null     object
 4   noise         3418 non-null    object
 5   dirt          124 non-null     object
 6   digging       6740 non-null    object
 7   capital       3490 non-null    object
 8   house_lines   31582 non-null   object
 9   clinics       280 non-null     object
 10  roads         3842 non-null    object
 11  parks         37 non-null      object
 12  transports    108 non-null     object
 13  city_places   3093 non-null    object
 14  live_area     103108 non-null  object
 15  streets       24 non-null      object
 16  trashes       26820 non-null   object
 17  factories     840 non-null     object
 18  markets       26 non-null      o

## Bad_gasstations

ID:754

**Некачественные автозаправки** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/754/)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=215808)

Набор данных содержит информацию о результатах контрольных мероприятий, проведённых Департаментом природопользования и охраны окружающей среды города Москвы в отношении автозаправочных станций с последующими лабораторными испытаниями отобранных на АЗС проб моторного топлива.

Экологические требования к качеству моторного топлива на территории города Москвы установлены постановлением Правительства Москвы от 28.12.2004 № 952-ПП «Об утверждении экологических требований к качеству моторного топлива при его обороте в городе Москве», также данным документом установлен перечень контролируемых Департаментом показателей моторного топлива (Приложение 3 к постановлению Правительства Москвы от 28.12.2004 № 952-ПП).

Перечень основных показателей моторного топлива, контролируемых Департаментом:

1. Автомобильные бензины:

– октановое число по исследовательскому методу;

– октановое число по моторному методу;

– концентрация свинца;

– концентрация серы;

– объёмная доля бензола;

- объёмная доля ароматических углеводородов.

2. Топливо дизельное:

– массовая доля полициклических ароматических углеводородов;

– концентрация серы;

- смазывающая способность;

- общее загрязнение.

Дата-сет позволяет получить подробную информацию об объектах реализации моторного топлива (АЗС), посмотреть их местоположение на карте, узнать  дату и результаты контрольных мероприятий. По каждой проверенной Департаментом АЗС можно посмотреть полное наименование юридического лица, осуществляющего деятельность по реализации моторного топлива на момент контрольных мероприятий, точный адрес АЗС.

Современная ситуация в России

Решением Комиссии Таможенного союза от 2 декабря 2015 года № 84 о внесении изменений в технический регламент таможенного союза «О требованиях к автомобильному и авиационному бензину, дизельному и судовому топливу, топливу для реактивных двигателей и мазуту» (ТР ТС 013/2011, на территории Российской Федерации с 1 января 2016 года запрещен выпуск в обращение и обращение дизельного топлива экологического класса Евро-4, а с 1 июля 2016 года выпуск в обращение и обращение автомобильного бензина экологического класса Евро-4.

Постановлением Правительства Российской Федерации от 29 декабря 2012 года        № 1474 «О применении некоторых актов Правительства Российской Федерации по вопросам технического регулирования и об органах государственного контроля (надзора) за соблюдением требований технического регламента таможенного союза «О требованиях к автомобильному и авиационному бензину, дизельному и судовому топливу, топливу для реактивных двигателей и топочному мазуту» установлено, что: Федеральное агентство по техническому регулированию и метрологии осуществляет государственный контроль (надзор) за соблюдением требований к автомобильному бензину, дизельному топливу, судовому топливу и мазуту, установленных Техническим регламентом Таможенного союза.

Современная ситуация по реализации моторного топлива в городе Москве

По итогам 2016 года в результате контрольных мероприятий на АЗС города Москвы выявлено 11 проб (11,2% от общего количества отобранных проб) моторного топлива, не соответствует утвержденным экологическим требованиям, в том числе 8 проб автомобильного бензина и 3 пробы дизельного топлива.

По данным фактам Департаментом виновные лица привлечены к административной ответственности в виде административных штрафов, также виновным лицам выданы предписания на устранение нарушений. В случае невыполнения выданных предписаний виновные лица также привлекаются Департаментом к административной ответственности в виде административных штрафов.

- АЗС на территории города Москвы контролируются Департаментом на предмет соответствия реализуемого моторного топлива экологическим требованиям с 2004 года в границах города до 1 июля 2012 года и с 1 июля 2012 года в современных границах города Москвы.

- В случае выявления фактов реализации на АЗС моторного топлива, не соответствующего экологическим требованиям, установленным на территории города Москвы, Вы вправе обратиться в Единую справочную службу Мэрии Москвы по тел.    8 (495) 777-77-77  либо письменно в Департамент (119019, Москва, ул. Новый Арбат, д. 11, корп. 1 либо через портал Департамента (http://eco.mos.ru/) с просьбой провести контрольные мероприятия в отношении указанных АЗС. При этом в обращении требуется указать причины обращения (перечислить признаки реализации моторного топлива, не соответствующего установленным экологическим требованиям), точный адрес расположения АЗС на территории города Москвы, Фамилию, Имя, Отчество и контактные данные (адрес заявителя и телефонный номер).

- В соответствии с положениями статьи 4.10 (Производство, оборот нефтепродуктов, не соответствующих экологическим требованиям) Кодекса города Москвы об административных правонарушениях установлена административная ответственность в виде административного штрафа на лиц, осуществляющих предпринимательскую деятельность без образования юридического лица, в размере 50 тысяч рублей; на юридических лиц – 400 тысяч рублей. При этом повторное совершение указанного административного правонарушения влечет наложение административного штрафа на юридических лиц в размере одного миллиона рублей.

Материал подготовлен на основе информации, полученной в результате контрольных мероприятий, проведённых Департаментом, и нормативных правовых актов Российской Федерации и города Москвы, http://data.mos.ru/.

In [48]:
bad_gasstations = get_dataset('754')
bad_gasstations['start'] = date(day=1, month=1, year=2017)
bad_gasstations['end'] = date.today().strftime('%d.%m.%Y')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          16 non-null     int64 
 1   FullName    16 non-null     object
 2   ShortName   16 non-null     object
 3   global_id   16 non-null     int64 
 4   AdmArea     16 non-null     object
 5   District    16 non-null     object
 6   Address     16 non-null     object
 7   Owner       16 non-null     object
 8   Violations  16 non-null     object
 9   TestDate    16 non-null     object
 10  geoData     16 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.5+ KB
None


In [49]:
ecology = fill_ecology(ecology, 'bad_gasstations', 'Owner', '', 'start', 'end', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185058 entries, 1 to 15
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   start            185058 non-null  object
 1   end              185058 non-null  object
 2   air              291 non-null     object
 3   mean             903 non-null     object
 4   noise            3418 non-null    object
 5   dirt             124 non-null     object
 6   digging          6740 non-null    object
 7   capital          3490 non-null    object
 8   house_lines      31582 non-null   object
 9   clinics          280 non-null     object
 10  roads            3842 non-null    object
 11  parks            37 non-null      object
 12  transports       108 non-null     object
 13  city_places      3093 non-null    object
 14  live_area        103108 non-null  object
 15  streets          24 non-null      object
 16  trashes          26820 non-null   object
 17  factories        84

## Buildgrounds

ID:2941

**Строй-площадки** [$\uparrow$ к таблице ключевых параметров $\uparrow$](#Table)

[Источник](https://data.mos.ru/opendata/2941)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=218838)

Набор данных «Объекты капитального строительства» содержит информацию о зданиях, строениях, сооружениях, объектах - строительство которых не завершено, за исключением временных построек, киосков, навесов и других подобных построек.

Дата-сет позволяет получить подробную информацию по каждому объекту: адресный ориентир, функциональное назначение, площадь застройки, разрешение на строительство.

Различают следующие виды объектов капитального строительства:

- Здания — результат строительства, представляющий собой объемную строительную систему, имеющую надземную и (или) подземную части, включающую в себя помещения, сети и системы инженерно-технического обеспечения и предназначенную для проживания и (или) деятельности людей, размещения производства, хранения продукции или содержания животных.
- Сооружения — результат строительства, представляющий собой объемную, плоскостную или линейную строительную систему, имеющую наземную, надземную и (или) подземную части, состоящую из несущих, а в отдельных случаях и ограждающих строительных конструкций и предназначенную для выполнения производственных процессов различного вида, хранения продукции, временного пребывания людей, перемещения людей и грузов.

Здания делятся на жилые и нежилые:

- Жилые здания состоят из комнат, а также помещений вспомогательного использования, предназначенных для удовлетворения гражданами бытовых и иных нужд, связанных с их проживанием в таком здании (жилые дома постоянного типа, общежития, приюты, дома маневренного фонда, дома-интернаты для престарелых и инвалидов, ветеранов, специальные дома для одиноких престарелых, детские дома, интернаты при школах и школы-интернаты, индивидуальные жилые дома, многоквартирные дома и др.);
- Нежилые здания - здания, назначением которых является создание условий для труда, социально-культурного обслуживания населения и хранения материальных ценностей: промышленные, сельскохозяйственные, коммерческие, административные, учебные, здравоохранения, другие.

#### Современная ситуация по объектам капитального строительства в России

Каждый месяц в России начинается строительство нового крупного объекта гражданского строительства.
В третьем квартале 2016 года объявлено о практическом начале нескольких проектов гражданского назначения, крупнейшие среди которых:
- Строительство жилого комплекса «Город-событие Лайково» в Московской области. Общий объем инвестиций в проект составляет 1000 млн. долларов. Строительство объекта планируется завершить в 2023 году.
- Строительство дистрибьюторского центра ИКЕА в Московской области. Объем инвестиций в проект составляет 170 млн рублей. Завершение строительства объекта намечено на 2018 год.
- Строительство комплекса общежитий «Перья» для Сибирского федерального университета в Красноярске. Общий объем инвестиций в проект – 40 млн рублей. Строительство объекта будет завершено в 2018 году.
- Строительство многофункционального музейного центра в Ленинградской области. Общий объем инвестиций в проект составляет 35 млн. долларов. Строительство объекта планируется завершить в 2019 году.
- Строительство бизнес-центра класса В+ «Охта Плаза». Объем инвестиций в строительство проекта составляет 33 млн рублей. Окончание строительства запланировано на 2018 год.

Как показывают данные аналитиков агентства INFOLine, в 2016 году происходит уменьшение объема инвестиций в проекты гражданского строительства. Но, как отмечают специалисты, во всех направлениях гражданского строительства, происходит и закладка новых объектов, а в некоторых увеличивается и объем работ.

#### Современная ситуация по объектам капитального строительства в Москве

Масштабы и темпы строительства в Москве - одни из самых высоких. По общей площади построенных зданий на душу населения и динамике показателя столица входит в число лидеров – 0,75 кв.м на чел., опережая Сингапур, Токио и Гонконг. За период 2010-2015 гг. реальный темп роста инвестиций в основной капитал на душу населения в национальной валюте составил более 4%, что значительно превышает аналогичный показатель для других 11 исследуемых городов. Москва занимает первое место по среднегодовым темпам роста показателя «Общий объем качественной офисной недвижимости на душу населения». За период 2009 - 2015 рост общей площади качественной офисной недвижимости на душу населения составил 22 %. Наблюдается устойчивая положительная тенденция к росту доступности жилья. Прирост индекса доступности жилья в 2015 по сравнению с 2009 составил 34%.
Для формирования комфортной городской среды с 2011 года в Москве реализуется Государственная программа «Градостроительная политика», которая системно увязывает стратегическое и территориальное планирование, обеспечивает координацию строительства жилой и коммерческой недвижимости с социальной, транспортной и инженерной инфраструктурой, объединяет нормативно-правовое, научно-методическое и информационное обеспечение в единый комплекс.

В числе приоритетов программы «Градостроительная политика»:

1. Определение приоритетов градостроительного развития города как столицы России, крупного международного финансового, научного, культурного центра.
2. Сохранение историко-архитектурного облика центра города, комплексное обустройство и развитие территории.
3. Формирование благоприятных условий для инвестиционной деятельности.
4. Совершенствование системы градостроительной деятельности.

«Зарядье» станет первым крупным парком, созданным в столице за последние 50 лет. Он соединит все пешеходные и туристические маршруты центра - от Варварки и Большого Москворецкого моста до Воробьевых гор. Здесь создадут уникальный ландшафт, построят филармонию, смотровые площадки, музеи и кафе. Парк вместит в себя все природное многообразие России: лес, степь, северные ландшафты и заливные луга. Эти зоны будут террасами спускаться с верхнего участка парка к его нижней части - с северо-востока на юго-запад. С Красной площади посетители попадут в березовую рощу, за которой вдоль Варварки протянутся низкорослые и стелющиеся растения северных ландшафтов. Это позволит сохранить вид на исторические памятники улицы. Ниже раскинется смешанный лес и большой луг, который плавно перейдет в крышу филармонии. А на юго-западе появятся пруды с сетью пешеходных деревянных настилов.

Проект онлайн-сервисов Москвы стал финалистом международной премии World Smart City Awards в категории «Город». Москва выставила на конкурс новый проект - концепцию городских сервисов для 12,5 млн жителей. Речь идет о внедрении централизованной системы управления и предоставления государственных услуг. С ее помощью горожанам стали доступны 200 онлайн и мобильных сервисов.

Материал подготовлен на основе информации http://dgp.mos.ru, https://stroi.mos.ru, http://marketing.rbc.ru.

In [50]:
buildgrounds = get_dataset('2941')
buildgrounds['end'] = date.today().strftime('%d.%m.%Y')
buildgrounds = buildgrounds[buildgrounds['GPZUDocumentDate'].notna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   global_id             771 non-null    int64  
 1   ID                    771 non-null    int64  
 2   ObjectName            771 non-null    object 
 3   AdmArea               771 non-null    object 
 4   District              771 non-null    object 
 5   ObjectAddress         771 non-null    object 
 6   Coordinates           771 non-null    object 
 7   ConsrtuctionWorkType  771 non-null    object 
 8   MainFunctional        771 non-null    object 
 9   SourceOfFinance       771 non-null    object 
 10  DevelopmentArea       751 non-null    float64
 11  FloorsAmount          771 non-null    object 
 12  ObjectStatus          771 non-null    object 
 13  DeveloperInfo         771 non-null    object 
 14  CadastralNumber       771 non-null    object 
 15  AreaCoordinates       7

In [51]:
ecology = fill_ecology(ecology, 'buildgrounds', 'GPZUDocumentDate', '', 'GPZUDocumentDate', 'end', 'geoData')
ecology.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185829 entries, 1 to 770
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   start            185829 non-null  object
 1   end              185829 non-null  object
 2   air              291 non-null     object
 3   mean             903 non-null     object
 4   noise            3418 non-null    object
 5   dirt             124 non-null     object
 6   digging          6740 non-null    object
 7   capital          3490 non-null    object
 8   house_lines      31582 non-null   object
 9   clinics          280 non-null     object
 10  roads            3842 non-null    object
 11  parks            37 non-null      object
 12  transports       108 non-null     object
 13  city_places      3093 non-null    object
 14  live_area        103108 non-null  object
 15  streets          24 non-null      object
 16  trashes          26820 non-null   object
 17  factories        8

In [56]:
build = get_dataset('3013')

Сохранено 17005 записей в 3013.gz
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17005 entries, 0 to 17004
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   UIDNFTN    17005 non-null  int64 
 1   NAME       17005 non-null  object
 2   FUNK_NAZ   17005 non-null  object
 3   ADRESS     17005 non-null  object
 4   AdmArea    17005 non-null  object
 5   District   17005 non-null  object
 6   DOC_NUM    17005 non-null  object
 7   DOC_DATE   17005 non-null  object
 8   global_id  17005 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.2+ MB
None


In [58]:
build.head()

,UIDNFTN,NAME,FUNK_NAZ,ADRESS,AdmArea,District,DOC_NUM,DOC_DATE,global_id
0,65538,ТЭО строительства административного здания,Административное здание,"Селезневская ул., вл.22",Центральный административный округ,Тверской район,372-4-97,04.11.1997,2757370
1,1101238,Корректировка проекта фасадов административно-управленческого здания,Административное здание,"Наметкина ул., д.14",Юго-Западный административный округ,район Черёмушки,559-4-06,29.09.2006,2757371
2,45565,ТЭО строительства индивидуального жилого дома,Индивидуальный жилой дом,"Митино, мкр.8А, корп.31",Северо-Западный административный округ,район Митино,210-2-97,02.12.1997,2757372
3,40082,Рабочий проект строительства временного торгового павильона,Торговля,"Шипиловский пр., площадка южного выхода из станции метро «Орехово»",Южный административный округ,район Орехово-Борисово Северное,286-4-98,06.07.1998,2757373
4,60212537,Корректировка проекта с использованием подземного пространства объекта «Универсальный спортивный комплекс»,Спортивное сооружение,"Зеленоград, Победы парк",Зеленоградский административный округ,район Савёлки,112-4-10/С,17.09.2010,2757374


In [62]:
build[build['ADRESS'].str.contains('Приорова')]

,UIDNFTN,NAME,FUNK_NAZ,ADRESS,AdmArea,District,DOC_NUM,DOC_DATE,global_id
1968,71957,ТЭО строительства многоэтажного гаража-стоянки,Гараж для хранения индивидуальных автомобилей,"улица Приорова, владение 16",Северный административный округ,район Коптево,122-5-97,30.09.1997,2823987
3986,624916,Рабочий проект (утверждаемая часть) строительства поликлиники на 240 посещений в смену с травмпунктом,Поликлиника,"Приорова ул., вл.10-12",Северный административный округ,район Коптево,707-4-03,17.12.2003,2826097
7693,6262932,Рабочий проект строительства поликлиники ГУН ЦИТО им. Н.Н. Приорова (2-я очередь),Объекты лечебно-оздоровительного назначения,"Приорова ул., вл.10-12",Северный административный округ,район Коптево,17-4-09/С,03.03.2009,2829919
8789,37174046,"Проект строительства, реконструкции и технического перевооружения комплекса зданий ФГУ «ЦИТО им. Н.Н. Приорова Росмедтехнологий»",Здравоохранение,"Приорова ул., вл.10",Северный административный округ,район Коптево,63-4-10/С,31.08.2010,2831054
9683,101438831,Проект строительства Центра физического и интеллектуального развития,Общественное,"Приорова ул., вл.9",Северный административный округ,район Коптево,13-4-13/С,01.03.2013,2831981
11768,252752427,"Архитектурно-градостроительное решение объекта «Реконструкция и техническое перевооружение производства с целью обеспечения выпуска импортозамещающих медицинских изделий для ортопедии, травматологии и протезирования ФГУП «ЦИТО»",Производственное,"Приорова ул., вл.10",Северный административный округ,район Коптево,75-4-16/С,04.03.2016,272840289
12039,238697785,Архитектурно-градостроительное решение объекта капитального строительства «Комплекс вспомогательных зданий и сооружений ФГБУ «ЦИТО им. Н.Н. Приорова» Министерства здравоохранения РФ»,Здравоохранение,"Приорова ул., вл.10",Северный административный округ,район Коптево,344-5-15/С,22.01.2016,272850227
12287,297624501,"Архитектурно-градостроительное решение объекта «Реконструкция и техническое перевооружение производства с целью обеспечения выпуска импортозамещающих медицинских изделий для ортопедии, травматологии и протезирования ФГУП «ЦИТО»",Производственное,"Приорова ул., вл.10",Северный административный округ,район Коптево,560-4-16/С,30.11.2016,282104127
14329,680961,Архитектурно-градостроительное решение объекта капитального строительства Реконструкция с пристройкой и надстройкой детского корпуса - 1 очередь ФГБУ НМИЦ ТО им. Н.Н. Приорова Минздрава России,Больница,"улица Приорова, владение 10",Северный административный округ,район Коптево,364-4-18/С,30.05.2018,880848084
14815,732319,"Архитектурно-градостроительное решение объекта капитального строительства Реконструкция и техническое перевооружение производства с целью обеспечения выпуска импортозамещающих медицинских изделий для ортопедии, травматологии и протезирования ФГУП ЦИТО, г. Москва",Производственное здание,"улица Приорова, владение 10, строение 7, 8",Северный административный округ,район Коптево,454-4-19/С,02.07.2019,1010405678


In [53]:
save_dataset('ecology')

Сохранено 185829 записей в ecology.gz
